# **Environment prepare**

In [ ]:
import ee
from google.cloud import storage

In [ ]:
!earthengine authenticate --auth_mode=notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ee.Initialize()

# **Prepare Data and Functions**

In [ ]:
import pandas as pd
import numpy as np
crop_type_replace=pd.read_excel("/content/drive/MyDrive/global_crop_dataset/crop_type_SPAM_42_correspond_5.xlsx")

In [ ]:
import collections
adm_name_replace=pd.read_csv("/content/drive/MyDrive/global_crop_dataset/FAO_ADM_replace_lxy_africa.csv",encoding= 'unicode_escape')
#crop_type_replace=pd.read_excel("/content/drive/MyDrive/global_crop_dataset/crop_type_SPAM_42_correspond_4_USDA.xlsx")
FAOSTAT_data=pd.read_csv("/content/drive/MyDrive/global_crop_dataset/FAOSTAT_africa_crop_1961-2022_CodeFAO.csv")

In [ ]:
irrigation_area=ee.Image('projects/ee-global-crop/assets/IrrigationArea_fao_gmia_aei_ha1')
pixelArea_10km=ee.Image('projects/ee-global-crop/assets/pixelArea_global')

In [ ]:
USDA_name_list=crop_type_replace['USDA_name'].tolist()
USDA_name_list=list(set(USDA_name_list))
USDA_name_list.remove(np.nan)
USDA_name_list.remove('BEANS')
print(USDA_name_list)

['RICE', 'SUGARCANE', 'TOBACCO', 'BARLEY', 'MILLET', 'CORN', 'LENTILS', 'SUNFLOWER', 'SORGHUM', 'CHICKPEAS', 'SUGARBEETS', 'COTTON', 'WHEAT', 'PEANUTS', 'SOYBEANS']


In [ ]:
continent=ee.FeatureCollection('projects/ee-lxy-data/assets/World_Continents_new')
africa=continent.filter(ee.Filter.eq('CONTINENT','Africa')).first()

In [ ]:
crop_name_list=['whea', 'rice', 'maiz', 'barl', 'pmil', 'smil', 'sorg', 'ocer', 'pota', 'swpo', 'yams', 'cass', 'orts', 'bean', 'chic', 'cowp', 'pige', 'lent', 'opul', 'soyb', 'grou', 'cnut', 'oilp', 'sunf', 'rape', 'sesa', 'ooil', 'sugc', 'sugb', 'cott', 'ofib', 'acof', 'rcof', 'coco', 'teas', 'toba', 'bana', 'plnt', 'trof', 'temf', 'vege', 'rest']
usda_name_list=['WHEAT','RICE','CORN','BARLEY','MILLET','MILLET','SORGHUM','','','','','','','BEANS','CHICKPEAS','','','LENTILS','','SOYBEANS','PEANUTS','','','SUNFLOWER','','','','SUGARCANE','SUGARBEETS','COTTON','','','','','','TOBACCO','','','','','','']
#CDL和STAT均为近年才有CHICKPEAS的数据，故这里删除
usda_name_list=['WHEAT','RICE','CORN','BARLEY','MILLET','MILLET','SORGHUM','','','','','','','BEANS','','','','LENTILS','','SOYBEANS','PEANUTS','','','SUNFLOWER','','','','SUGARCANE','SUGARBEETS','COTTON','','','','','','TOBACCO','','','','','','']

In [ ]:
geo_adm0=ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")
geo_adm1=ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
geo_adm2=ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
usa_adm0=geo_adm0.filter(ee.Filter.eq('ADM0_CODE',259))
usa_adm1=geo_adm1.filter(ee.Filter.eq('ADM0_CODE',259))
usa_adm2=geo_adm2.filter(ee.Filter.eq('ADM0_CODE',259))
cdl=ee.ImageCollection("USDA/NASS/CDL")

In [ ]:
chn_adm1=geo_adm1.filter(ee.Filter.eq('ADM0_CODE',147295))

In [ ]:
china_crop_list=['大豆','稻谷','豆类','甘蔗','花生','烤烟','麻类','棉花','蔬菜','薯类','甜菜','小麦','油菜籽','玉米']
#spam_china_crop_list=['soyb','rice',['bean','chic','cowp','lent','opul','pige'],'sugc','grou','toba','ofib','cott','vege',['cass','orts','pota','swpo','yams'],'sugb','whea','rape','maiz']
spam_china_crop_list=[['soyb'],['rice'],['bean','chic','cowp','lent','opul','pige'],['sugc'],['grou'],['toba'],['ofib'],['cott'],['vege'],['cass','orts','pota','swpo','yams'],['sugb'],['whea'],['rape'],['maiz']]
multi_year_data_training=[1,1,0,1,0,0,0,1,0,0,0,1,1,1]
train_year_list_list=[[2017,2018,2019],[2017,2018,2019,2020],[],[2016,2017,2018,2019,2020],[],[],[],[2018,2019,2020,2021],[],[],[],[2016,2017,2018,2019,2020],[2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022],[2017,2018,2019,2020,2021]]

In [ ]:
spam_china_crop_list_1=sum(spam_china_crop_list,[])
print(spam_china_crop_list_1)

['soyb', 'rice', 'bean', 'chic', 'cowp', 'lent', 'opul', 'pige', 'sugc', 'grou', 'toba', 'ofib', 'cott', 'vege', 'cass', 'orts', 'pota', 'swpo', 'yams', 'sugb', 'whea', 'rape', 'maiz']


# **Universial functions**

In [ ]:
def get_unitscale_img(image,scale,image_name,gem):
  image=image.rename('band')
  name='band'
  minMax = image.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': gem.geometry(),
    'scale': scale,
    'maxPixels': 10e12,
    'tileScale': 16
  });

  unitScale_img = ee.Image(ee.Algorithms.If(
    ee.Number(minMax.get(name+'_max')).getInfo()!=ee.Number(minMax.get(name+'_min')).getInfo(),
    image.unitScale(ee.Number(minMax.get(name+'_min')), ee.Number(minMax.get(name+'_max'))).rename(image_name),
    ee.Image(ee.Number(minMax.get(name+'_max')).getInfo())
  ))
  #image.unitScale(ee.Number(minMax.get(name+'_min')), ee.Number(minMax.get(name+'_max'))).rename(image_name)
  unitScale_img = unitScale_img.rename(image_name)
  return unitScale_img

In [ ]:
def produce_suit_layer(crop_id,year,system):
  irrigation_area=ee.Image('projects/ee-global-crop/assets/IrrigationArea_fao_gmia_aei_ha1')
  crop_list_irri=['alf','ban','brl','cit','coc','cof','con','cot','csv','flx','grd','jtr','mis','olp','olv','rcg','rsd','sfl','soy','spo','sub','suc','swg','tea','tob','whe','wpo']


  crop_type_selected=crop_type_replace.loc[crop_type_replace['crop_id'] == crop_id]
  SPAM_longName=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("SPAM_longName")])
  SPAM_shortName=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("SPAM_shortName")])
  FAO_code_list=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("FAO_code")]).split(',')
  GAEZ_longName_list=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("GAEZ_longName")]).split(',')
  GAEZ_shortName_list=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("GAEZ_shortName")]).split(',')
  '''
  print(SPAM_longName)
  print(SPAM_shortName)
  print(FAO_code_list)
  print(GAEZ_longName_list)
  print(GAEZ_shortName_list)
  '''

  if SPAM_shortName!='rice':
    GAEZ_shortName=GAEZ_shortName_list[0]
    suit_8110H=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_8110H_su/suHr_'+GAEZ_shortName)
    suit_8110H=suit_8110H.updateMask(suit_8110H.neq(-1)).unmask(0)
    suit_7100H=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_7100H_su/suHr_'+GAEZ_shortName)
    suit_7100H=suit_7100H.updateMask(suit_7100H.neq(-1)).unmask(0)
    suit_6190H=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_6190H_su/suHr_'+GAEZ_shortName)
    suit_6190H=suit_6190H.updateMask(suit_6190H.neq(-1)).unmask(0)
    suit_8110L=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_8110L_su/suLr_'+GAEZ_shortName)
    suit_8110L=suit_8110L.updateMask(suit_8110L.neq(-1)).unmask(0)
    suit_7100L=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_7100L_su/suLr_'+GAEZ_shortName)
    suit_7100L=suit_7100L.updateMask(suit_7100L.neq(-1)).unmask(0)
    suit_6190L=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_6190L_su/suLr_'+GAEZ_shortName)
    suit_6190L=suit_6190L.updateMask(suit_6190L.neq(-1)).unmask(0)
    if GAEZ_shortName in crop_list_irri:
      suit_8110I=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_8110I_su/suHi_'+GAEZ_shortName)
      suit_8110I=suit_8110I.updateMask(suit_8110I.neq(-1)).unmask(0)
      suit_7100I=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_7100I_su/suHi_'+GAEZ_shortName)
      suit_7100I=suit_7100I.updateMask(suit_7100I.neq(-1)).unmask(0)
      suit_6190I=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_6190I_su/suHi_'+GAEZ_shortName)
      suit_6190I=suit_6190I.updateMask(suit_6190I.neq(-1)).unmask(0)
    else:
      suit_8110I=ee.ImageCollection([suit_8110H,suit_8110L]).max()
      suit_7100I=ee.ImageCollection([suit_7100H,suit_7100L]).max()
      suit_6190I=ee.ImageCollection([suit_6190H,suit_6190L]).max()
  else:
    GAEZ_shortName_1='rcw'
    GAEZ_shortName_2='rcd'
    GAEZ_shortName_3='rcg'

    suit_8110H_1=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_8110H_su/suHr_'+GAEZ_shortName_1)
    suit_8110H_1=suit_8110H_1.updateMask(suit_8110H_1.neq(-1)).unmask(0).updateMask(irrigation_area)
    suit_7100H_1=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_7100H_su/suHr_'+GAEZ_shortName_1)
    suit_7100H_1=suit_7100H_1.updateMask(suit_7100H_1.neq(-1)).unmask(0).updateMask(irrigation_area)
    suit_6190H_1=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_6190H_su/suHr_'+GAEZ_shortName_1)
    suit_6190H_1=suit_6190H_1.updateMask(suit_6190H_1.neq(-1)).unmask(0).updateMask(irrigation_area)
    suit_8110L_1=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_8110L_su/suLr_'+GAEZ_shortName_1)
    suit_8110L_1=suit_8110L_1.updateMask(suit_8110L_1.neq(-1)).unmask(0).updateMask(irrigation_area)
    suit_7100L_1=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_7100L_su/suLr_'+GAEZ_shortName_1)
    suit_7100L_1=suit_7100L_1.updateMask(suit_7100L_1.neq(-1)).unmask(0).updateMask(irrigation_area)
    suit_6190L_1=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_6190L_su/suLr_'+GAEZ_shortName_1)
    suit_6190L_1=suit_6190L_1.updateMask(suit_6190L_1.neq(-1)).unmask(0).updateMask(irrigation_area)

    suit_8110H_2=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_8110H_su/suHr_'+GAEZ_shortName_2)
    suit_8110H_2=suit_8110H_2.updateMask(suit_8110H_2.neq(-1)).unmask(0).updateMask(irrigation_area.lte(0))
    suit_7100H_2=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_7100H_su/suHr_'+GAEZ_shortName_2)
    suit_7100H_2=suit_7100H_2.updateMask(suit_7100H_2.neq(-1)).unmask(0).updateMask(irrigation_area.lte(0))
    suit_6190H_2=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_6190H_su/suHr_'+GAEZ_shortName_2)
    suit_6190H_2=suit_6190H_2.updateMask(suit_6190H_2.neq(-1)).unmask(0).updateMask(irrigation_area.lte(0))
    suit_8110L_2=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_8110L_su/suLr_'+GAEZ_shortName_2)
    suit_8110L_2=suit_8110L_2.updateMask(suit_8110L_2.neq(-1)).unmask(0).updateMask(irrigation_area.lte(0))
    suit_7100L_2=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_7100L_su/suLr_'+GAEZ_shortName_2)
    suit_7100L_2=suit_7100L_2.updateMask(suit_7100L_2.neq(-1)).unmask(0).updateMask(irrigation_area.lte(0))
    suit_6190L_2=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_6190L_su/suLr_'+GAEZ_shortName_2)
    suit_6190L_2=suit_6190L_2.updateMask(suit_6190L_2.neq(-1)).unmask(0).updateMask(irrigation_area.lte(0))

    suit_8110H_3=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_8110H_su/suHr_'+GAEZ_shortName_3)
    suit_8110H_3=suit_8110H_3.updateMask(suit_8110H_3.neq(-1)).unmask(0)
    suit_7100H_3=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_7100H_su/suHr_'+GAEZ_shortName_3)
    suit_7100H_3=suit_7100H_3.updateMask(suit_7100H_3.neq(-1)).unmask(0)
    suit_6190H_3=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_6190H_su/suHr_'+GAEZ_shortName_3)
    suit_6190H_3=suit_6190H_3.updateMask(suit_6190H_3.neq(-1)).unmask(0)
    suit_8110L_3=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_8110L_su/suLr_'+GAEZ_shortName_3)
    suit_8110L_3=suit_8110L_3.updateMask(suit_8110L_3.neq(-1)).unmask(0)
    suit_7100L_3=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_7100L_su/suLr_'+GAEZ_shortName_3)
    suit_7100L_3=suit_7100L_3.updateMask(suit_7100L_3.neq(-1)).unmask(0)
    suit_6190L_3=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_6190L_su/suLr_'+GAEZ_shortName_3)
    suit_6190L_3=suit_6190L_3.updateMask(suit_6190L_3.neq(-1)).unmask(0)
    suit_8110I_3=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_8110I_su/suHi_'+GAEZ_shortName_3)
    suit_8110I_3=suit_8110I_3.updateMask(suit_8110I_3.neq(-1)).unmask(0)
    suit_7100I_3=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_7100I_su/suHi_'+GAEZ_shortName_3)
    suit_7100I_3=suit_7100I_3.updateMask(suit_7100I_3.neq(-1)).unmask(0)
    suit_6190I_3=ee.Image('projects/ee-global-crop/assets/GAEZ_suitIndex_6190I_su/suHi_'+GAEZ_shortName_3)
    suit_6190I_3=suit_6190I_3.updateMask(suit_6190I_3.neq(-1)).unmask(0)

    suit_8110H=suit_8110H_1.unmask(suit_8110H_2)
    suit_7100H=suit_7100H_1.unmask(suit_7100H_2)
    suit_6190H=suit_6190H_1.unmask(suit_6190H_2)
    suit_8110L=suit_8110L_1.unmask(suit_8110L_2)
    suit_7100L=suit_7100L_1.unmask(suit_7100L_2)
    suit_6190L=suit_6190L_1.unmask(suit_6190L_2)
    suit_8110I=suit_8110I_3
    suit_7100I=suit_7100I_3
    suit_6190I=suit_6190I_3

  if year<=1970:
    if system=='i':
      suit=suit_6190I
    elif system=='h':
      suit=suit_6190H
    else:
      suit=suit_6190L
  elif year<=1980:
    if system=='i':
      suit=ee.ImageCollection([suit_6190I,suit_7100I]).mean()
    elif system=='h':
      suit=ee.ImageCollection([suit_6190H,suit_7100H]).mean()
    else:
      suit=ee.ImageCollection([suit_6190L,suit_7100L]).mean()
  elif year<=1990:
    if system=='i':
      suit=ee.ImageCollection([suit_6190I,suit_7100I,suit_8110I]).mean()
    elif system=='h':
      suit=ee.ImageCollection([suit_6190H,suit_7100H,suit_8110H]).mean()
    else:
      suit=ee.ImageCollection([suit_6190L,suit_7100L,suit_8110L]).mean()
  elif year<=2000:
    if system=='i':
      suit=ee.ImageCollection([suit_7100I,suit_8110I]).mean()
    elif system=='h':
      suit=ee.ImageCollection([suit_7100H,suit_8110H]).mean()
    else:
      suit=ee.ImageCollection([suit_7100L,suit_8110L]).mean()
  elif year<=2010:
    if system=='i':
      suit=suit_8110I
    if system=='h':
      suit=suit_8110H
    else:
      suit=suit_8110L
  else:#2010-2022
    if system=='i':
      suit=suit_8110I
    if system=='h':
      suit=suit_8110H
    else:
      suit=suit_8110L

  return suit

def produce_cropland_layer(year):
  cropland=ee.Image('projects/ee-global-crop/assets/cropland_GSCM_spam_Global-cropland-percentage-map_1km')
  FROM_GLC_Plus=ee.ImageCollection('projects/ee-fgp-1km-prop/assets/fgp-1km-prop')

  plus_2010=FROM_GLC_Plus.filter(ee.Filter.stringContains('system:index','2010')).first()
  cropland_2010=plus_2010.select('crop')

  if year>=1982:
    plus_y=FROM_GLC_Plus.filter(ee.Filter.stringContains('system:index',str(year))).first()
    cropland_y0=plus_y.select('crop')
    '''
    if year>2010:
      plus_y_1=FROM_GLC_Plus.filter(ee.Filter.stringContains('system:index',str(year+1))).first()
    elif year<2010:
      plus_y_1=FROM_GLC_Plus.filter(ee.Filter.stringContains('system:index',str(year-1))).first()
    else:#year==2010
      plus_y_1=FROM_GLC_Plus.filter(ee.Filter.stringContains('system:index',str(year))).first()
    '''
    cropland_y=cropland_y0.subtract(cropland_2010).add(cropland)
  else:
    if year>=1980:
      cropland_y0=ee.Image("users/cbw/globalCropland/globalCropland_1980CE")
    elif year>=1970:
      cropland_y0=ee.Image("users/cbw/globalCropland/globalCropland_1970CE")
    else:
      cropland_y0=ee.Image("users/cbw/globalCropland/globalCropland_1960CE")
    cropland_y=cropland_y0.subtract(cropland_2010).add(cropland)
  cropland_y=cropland_y.where(cropland_y.gt(1),1)
  cropland_y=cropland_y.where(cropland_y.lt(0),0)

  return cropland_y

def get_cropland_layer(year):
  if year>=1982:
    cropland_y=ee.Image("projects/ee-global-crop/assets/Cropland_global_annual_10km/"+"cropland_img_10km_"+str(year))
    if year>2021:
      cropland_y=ee.Image("projects/ee-global-crop/assets/Cropland_global_annual_10km/"+"cropland_img_10km_"+str(2021))
  else:
    if year>=1980:
      cropland_y=ee.Image("projects/ee-global-crop/assets/Cropland_global_annual_10km/"+"cropland_img_10km_"+str(1980))
    elif year>=1970:
      cropland_y=ee.Image("projects/ee-global-crop/assets/Cropland_global_annual_10km/"+"cropland_img_10km_"+str(1970))
    else:
      cropland_y=ee.Image("projects/ee-global-crop/assets/Cropland_global_annual_10km/"+"cropland_img_10km_"+str(1960))
  return cropland_y

In [ ]:
def produce_PotYield_layer(crop_id,year,system):
  irrigation_area=ee.Image('projects/ee-global-crop/assets/IrrigationArea_fao_gmia_aei_ha1')
  crop_list_irri=['alf','ban','brl','cit','coc','cof','con','cot','csv','flx','grd','jtr','mis','olp','olv','rcg','rsd','sfl','soy','spo','sub','suc','swg','tea','tob','whe','wpo']

  crop_type_selected=crop_type_replace.loc[crop_type_replace['crop_id'] == crop_id]
  SPAM_longName=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("SPAM_longName")])
  SPAM_shortName=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("SPAM_shortName")])
  FAO_code_list=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("FAO_code")]).split(',')
  GAEZ_longName_list=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("GAEZ_longName")]).split(',')
  GAEZ_shortName_list=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("GAEZ_shortName")]).split(',')
  '''
  print(SPAM_longName)
  print(SPAM_shortName)
  print(FAO_code_list)
  print(GAEZ_longName_list)
  print(GAEZ_shortName_list)
  '''

  if SPAM_shortName!='rice':
    GAEZ_shortName=GAEZ_shortName_list[0]
    yield_8110H=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_8110H/ycHr_'+GAEZ_shortName)
    yield_8110H=yield_8110H.updateMask(yield_8110H.neq(-1)).unmask(0)
    yield_7100H=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_7100H/ycHr_'+GAEZ_shortName)
    yield_7100H=yield_7100H.updateMask(yield_7100H.neq(-1)).unmask(0)
    yield_6190H=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_6190H/ycHr_'+GAEZ_shortName)
    yield_6190H=yield_6190H.updateMask(yield_6190H.neq(-1)).unmask(0)
    yield_8110L=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_8110L/ycLr_'+GAEZ_shortName)
    yield_8110L=yield_8110L.updateMask(yield_8110L.neq(-1)).unmask(0)
    yield_7100L=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_7100L/ycLr_'+GAEZ_shortName)
    yield_7100L=yield_7100L.updateMask(yield_7100L.neq(-1)).unmask(0)
    yield_6190L=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_6190L/ycLr_'+GAEZ_shortName)
    yield_6190L=yield_6190L.updateMask(yield_6190L.neq(-1)).unmask(0)

    if GAEZ_shortName in crop_list_irri:
      yield_8110I=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_8110I/ycHi_'+GAEZ_shortName)
      yield_8110I=yield_8110I.updateMask(yield_8110I.neq(-1)).unmask(0)
      yield_7100I=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_7100I/ycHi_'+GAEZ_shortName)
      yield_7100I=yield_7100I.updateMask(yield_7100I.neq(-1)).unmask(0)
      yield_6190I=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_6190I/ycHi_'+GAEZ_shortName)
      yield_6190I=yield_6190I.updateMask(yield_6190I.neq(-1)).unmask(0)
    else:
      yield_8110I=ee.ImageCollection([yield_8110H,yield_8110L]).max()
      yield_7100I=ee.ImageCollection([yield_7100H,yield_7100L]).max()
      yield_6190I=ee.ImageCollection([yield_6190H,yield_6190L]).max()

  else:
    GAEZ_shortName_1='rcw'
    GAEZ_shortName_2='rcd'
    GAEZ_shortName_3='rcg'

    yield_8110H_1=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_8110H/ycHr_'+GAEZ_shortName_1)
    yield_8110H_1=yield_8110H_1.updateMask(yield_8110H_1.neq(-1)).unmask(0).updateMask(irrigation_area)
    yield_7100H_1=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_7100H/ycHr_'+GAEZ_shortName_1)
    yield_7100H_1=yield_7100H_1.updateMask(yield_7100H_1.neq(-1)).unmask(0).updateMask(irrigation_area)
    yield_6190H_1=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_6190H/ycHr_'+GAEZ_shortName_1)
    yield_6190H_1=yield_6190H_1.updateMask(yield_6190H_1.neq(-1)).unmask(0).updateMask(irrigation_area)
    yield_8110L_1=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_8110L/ycLr_'+GAEZ_shortName_1)
    yield_8110L_1=yield_8110L_1.updateMask(yield_8110L_1.neq(-1)).unmask(0).updateMask(irrigation_area)
    yield_7100L_1=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_7100L/ycLr_'+GAEZ_shortName_1)
    yield_7100L_1=yield_7100L_1.updateMask(yield_7100L_1.neq(-1)).unmask(0).updateMask(irrigation_area)
    yield_6190L_1=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_6190L/ycLr_'+GAEZ_shortName_1)
    yield_6190L_1=yield_6190L_1.updateMask(yield_6190L_1.neq(-1)).unmask(0).updateMask(irrigation_area)

    yield_8110H_2=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_8110H/ycHr_'+GAEZ_shortName_2)
    yield_8110H_2=yield_8110H_2.updateMask(yield_8110H_2.neq(-1)).unmask(0).updateMask(irrigation_area.lte(0))
    yield_7100H_2=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_7100H/ycHr_'+GAEZ_shortName_2)
    yield_7100H_2=yield_7100H_2.updateMask(yield_7100H_2.neq(-1)).unmask(0).updateMask(irrigation_area.lte(0))
    yield_6190H_2=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_6190H/ycHr_'+GAEZ_shortName_2)
    yield_6190H_2=yield_6190H_2.updateMask(yield_6190H_2.neq(-1)).unmask(0).updateMask(irrigation_area.lte(0))
    yield_8110L_2=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_8110L/ycLr_'+GAEZ_shortName_2)
    yield_8110L_2=yield_8110L_2.updateMask(yield_8110L_2.neq(-1)).unmask(0).updateMask(irrigation_area.lte(0))
    yield_7100L_2=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_7100L/ycLr_'+GAEZ_shortName_2)
    yield_7100L_2=yield_7100L_2.updateMask(yield_7100L_2.neq(-1)).unmask(0).updateMask(irrigation_area.lte(0))
    yield_6190L_2=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_6190L/ycLr_'+GAEZ_shortName_2)
    yield_6190L_2=yield_6190L_2.updateMask(yield_6190L_2.neq(-1)).unmask(0).updateMask(irrigation_area.lte(0))

    yield_8110H_3=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_8110H/ycHr_'+GAEZ_shortName_3)
    yield_8110H_3=yield_8110H_3.updateMask(yield_8110H_3.neq(-1)).unmask(0)
    yield_7100H_3=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_7100H/ycHr_'+GAEZ_shortName_3)
    yield_7100H_3=yield_7100H_3.updateMask(yield_7100H_3.neq(-1)).unmask(0)
    yield_6190H_3=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_6190H/ycHr_'+GAEZ_shortName_3)
    yield_6190H_3=yield_6190H_3.updateMask(yield_6190H_3.neq(-1)).unmask(0)
    yield_8110L_3=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_8110L/ycLr_'+GAEZ_shortName_3)
    yield_8110L_3=yield_8110L_3.updateMask(yield_8110L_3.neq(-1)).unmask(0)
    yield_7100L_3=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_7100L/ycLr_'+GAEZ_shortName_3)
    yield_7100L_3=yield_7100L_3.updateMask(yield_7100L_3.neq(-1)).unmask(0)
    yield_6190L_3=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_6190L/ycLr_'+GAEZ_shortName_3)
    yield_6190L_3=yield_6190L_3.updateMask(yield_6190L_3.neq(-1)).unmask(0)
    yield_8110I_3=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_8110I/ycHi_'+GAEZ_shortName_3)
    yield_8110I_3=yield_8110I_3.updateMask(yield_8110I_3.neq(-1)).unmask(0)
    yield_7100I_3=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_7100I/ycHi_'+GAEZ_shortName_3)
    yield_7100I_3=yield_7100I_3.updateMask(yield_7100I_3.neq(-1)).unmask(0)
    yield_6190I_3=ee.Image('projects/ee-global-crop/assets/GAEZ_PotYield_6190I/ycHi_'+GAEZ_shortName_3)
    yield_6190I_3=yield_6190I_3.updateMask(yield_6190I_3.neq(-1)).unmask(0)

    yield_8110H=yield_8110H_1.unmask(yield_8110H_2)
    yield_7100H=yield_7100H_1.unmask(yield_7100H_2)
    yield_6190H=yield_6190H_1.unmask(yield_6190H_2)
    yield_8110L=yield_8110L_1.unmask(yield_8110L_2)
    yield_7100L=yield_7100L_1.unmask(yield_7100L_2)
    yield_6190L=yield_6190L_1.unmask(yield_6190L_2)
    yield_8110I=yield_8110I_3
    yield_7100I=yield_7100I_3
    yield_6190I=yield_6190I_3

  if year<=1970:
    if system=='i':
      yield_1=yield_6190I
    elif system=='h':
      yield_1=yield_6190H
    else:
      yield_1=yield_6190L
  elif year<=1980:
    if system=='i':
      yield_1=ee.ImageCollection([yield_6190I,yield_7100I]).mean()
    elif system=='h':
      yield_1=ee.ImageCollection([yield_6190H,yield_7100H]).mean()
    else:
      yield_1=ee.ImageCollection([yield_6190L,yield_7100L]).mean()
  elif year<=1990:
    if system=='i':
      yield_1=ee.ImageCollection([yield_6190I,yield_7100I,yield_8110I]).mean()
    elif system=='h':
      yield_1=ee.ImageCollection([yield_6190H,yield_7100H,yield_8110H]).mean()
    else:
      yield_1=ee.ImageCollection([yield_6190L,yield_7100L,yield_8110L]).mean()
  elif year<=2000:
    if system=='i':
      yield_1=ee.ImageCollection([yield_7100I,yield_8110I]).mean()
    elif system=='h':
      yield_1=ee.ImageCollection([yield_7100H,yield_8110H]).mean()
    else:
      yield_1=ee.ImageCollection([yield_7100L,yield_8110L]).mean()
  elif year<=2010:
    if system=='i':
      yield_1=yield_8110I
    if system=='h':
      yield_1=yield_8110H
    else:
      yield_1=yield_8110L
  else:#2010-2022
    if system=='i':
      yield_1=yield_8110I
    if system=='h':
      yield_1=yield_8110H
    else:
      yield_1=yield_8110L

  return yield_1

In [ ]:
def get_harvest_area_max(year):
  cropland=get_cropland_layer(year)
  if year<=2019 and year>=2001:
    GCI=ee.Image('projects/ee-global-crop/assets/Crop_intensity_GCI_2000-19_10km_new/'+"GCI_10km"+'_'+str(year))
  elif year<=2000:
    GCI=ee.Image('projects/ee-global-crop/assets/Crop_intensity_GCI_2000-19_10km_new/'+"GCI_10km"+'_'+str(2001))
  else:
    GCI=ee.Image('projects/ee-global-crop/assets/Crop_intensity_GCI_2000-19_10km_new/'+"GCI_10km"+'_'+str(2019))
  GCI=GCI.where(GCI.eq(0).And(cropland.neq(0)),1)
  GCI=GCI.where(GCI.lt(1).And(cropland.neq(0)),1)
  max=cropland.multiply(GCI).multiply(pixelArea_10km).divide(10000)
  return max

# **Define Functions-CHN**

In [ ]:
chn_adm1=geo_adm1.filter(ee.Filter.eq('ADM0_CODE',147295))
fishnet=ee.FeatureCollection('projects/ee-lxy-data/assets/world_fishnet_2du')
chn_fishnet=fishnet.filterBounds(chn_adm1)
chn_fishnet_Id_list=chn_fishnet.aggregate_array('Id').getInfo()

In [ ]:
china_crop_list=['大豆','稻谷','豆类','甘蔗','花生','烤烟','麻类','棉花','蔬菜','薯类','甜菜','小麦','油菜籽','玉米']
#spam_china_crop_list=['soyb','rice',['bean','chic','cowp','lent','opul','pige'],'sugc','grou','toba','ofib','cott','vege',['cass','orts','pota','swpo','yams'],'sugb','whea','rape','maiz']
spam_china_crop_list=[['soyb'],['rice'],['bean','chic','cowp','lent','opul','pige'],['sugc'],['grou'],['toba'],['ofib'],['cott'],['vege'],['cass','orts','pota','swpo','yams'],['sugb'],['whea'],['rape'],['maiz']]
multi_year_data_training=[1,1,0,1,0,0,0,1,0,0,0,1,1,1]

In [ ]:
multiyear_crop_list=['soyb','rice','sugc','cott','whea','rape','maiz']

In [ ]:
def reduceResolution_10km(img,p,scale):

  img=img.setDefaultProjection(**{
    'crs': p,
    'crsTransform':None,
    'scale':scale
  })

  img_10km = img.reduceResolution(**{
    'reducer': ee.Reducer.mean(),
    'bestEffort':True,
    'maxPixels':60000,
  }).reproject(p,None,10000)

  return img_10km

def img_rename(img):
  return img.rename('b1').toInt()

def get_china_spam_2010_map_multiyear(crop_name,year):
  for crop_china_name in china_crop_list:
    #crop_name=china_crop_list[id]
    id=china_crop_list.index(crop_china_name)
    spam_list=spam_china_crop_list[id]
    multi_year_bool=multi_year_data_training[id]

    spam_list_len=len(spam_list)
    if spam_list_len!=1:
      continue

    SPAM_shortName=spam_list[0]
    if crop_name != SPAM_shortName:
      continue
    if multi_year_bool==0:
      continue

    if SPAM_shortName=='soyb':
      #cdl_2019=ee.Image('users/horacebbbb/HLJ_soybean/CDL2019_clip') #尤南山结果 0:rice,1:maize,2:soybean,3:others
      if year!=2019:
        img_m=ee.Image('users/horacebbbb/HLJ_soybean/CDL'+str(year)+'_clip').eq(2)
        img_10km=reduceResolution_10km(img_m,img_m.projection(),10)
      else:
        img_m=ee.Image('projects/glad/crop/chn_maize_soy_10m_2019').eq(2)
        img_10km=reduceResolution_10km(img_m,img_m.projection(),10)

    elif SPAM_shortName=='rice':
      '''
      img_c=ee.ImageCollection('projects/ee-china-crop/assets/CHNCropArea1km').filter(ee.Filter.stringContains('system:index','Rice')).filter(ee.Filter.stringContains('system:index',str(year)))
      img_m=img_c.first()
      img_10km=reduceResolution_10km(img_m,img_m.projection(),1000)
      '''

      img_c=ee.ImageCollection('projects/ee-china-crop/assets/yuan_china_single_rice_17-21').filter(ee.Filter.stringContains('system:index',str(year)))
      img_early=ee.ImageCollection('projects/ee-china-crop/assets/yuan_rice_'+str(year)+'_LR')
      img_late=ee.ImageCollection('projects/ee-china-crop/assets/yuan_rice_'+str(year)+'_ER')
      img_c=img_c.merge(img_early).merge(img_late).map(lambda img: img_rename(img))
      img_m=img_c.qualityMosaic('b1')
      img_1=img_c.first()
      p1=img_1.projection()

      '''
      img_m=img_m.setDefaultProjection({
        'crs': p1,
        'crsTransform':None,
        'scale':10
      })
      '''
      img_10km=reduceResolution_10km(img_m,p1,10)

      '''
      img_m=ee.Image('projects/ee-global-crop/assets/asia_rice_2000-2020/'+str(year))
      img_10km=reduceResolution_10km(img_m,img_m.projection(),500)
      '''

    elif SPAM_shortName=='sugc':
      img_c=ee.ImageCollection('projects/ee-china-crop/assets/sugarcane_china').filter(ee.Filter.stringContains('system:index',str(year))).map(lambda img: img_rename(img))
      img_m=img_c.qualityMosaic('b1')
      #img_m=img_c.mosaic()
      img_1=img_c.first()
      p1=img_1.projection()
      '''
      img_m=img_m.setDefaultProjection({
        'crs': p1,
        'crsTransform':None,
        'scale':30
      })
      '''
      img_10km=reduceResolution_10km(img_m,p1,30)
    elif SPAM_shortName=='cott':
      img_c=ee.ImageCollection('projects/ee-china-crop/assets/china_cotton_2018-2021').filter(ee.Filter.stringContains('system:index',str(year))).map(lambda img: img_rename(img))
      img_m=img_c.qualityMosaic('b1')
      #img_m=img_c.mosaic()
      img_1=img_c.first()
      p1=img_1.projection()

      '''
      img_m=img_m.setDefaultProjection({
        'crs': p1,
        'crsTransform':None,
        'scale':10
      })
      '''
      img_10km=reduceResolution_10km(img_m,p1,10)
    elif SPAM_shortName=='whea':
      '''
      img_c=ee.ImageCollection('projects/ee-china-crop/assets/CHNCropArea1km').filter(ee.Filter.stringContains('system:index','Wheat')).filter(ee.Filter.stringContains('system:index',str(year)))
      img_m=img_c.first()
      img_10km=reduceResolution_10km(img_m,img_m.projection(),1000)
      '''
      img_m=ee.Image('projects/ee-china-crop/assets/yuan_china_wheat_'+str(year))
      img_10km=reduceResolution_10km(img_m,img_m.projection(),30)


    elif SPAM_shortName=='rape':
      #img_m=ee.Image('users/Conala/ChinaRapeseedMap2017-2021/China_Rapeseed_'+str(year))
      img_m=ee.Image('projects/ee-china-crop/assets/china_rapeseed_2000-2022/'+str(year))
      img_10km=reduceResolution_10km(img_m,img_m.projection(),30)

    elif SPAM_shortName=='maiz':
      '''
      img_c=ee.ImageCollection('projects/ee-china-crop/assets/CHNCropArea1km').filter(ee.Filter.stringContains('system:index','Maize')).filter(ee.Filter.stringContains('system:index',str(year)))
      img_m=img_c.first()
      img_10km=reduceResolution_10km(img_m,img_m.projection(),1000)
      '''
      #img_c=ee.ImageCollection('projects/ee-china-crop/assets/ChinaMaize10_17-21').filter(ee.Filter.stringContains('system:index',str(year))).map(lambda img: img_rename(img))
      img_c=ee.ImageCollection('projects/ee-china-crop/assets/china_maize_2001-2020').filter(ee.Filter.stringContains('system:index',str(year))).map(lambda img: img_rename(img))
      img_m=img_c.qualityMosaic('b1')
      #img_m=img_c.mosaic()
      img_1=img_c.first()
      p1=img_1.projection()

      '''
      img_m=img_m.setDefaultProjection({
        'crs': p1,
        'crsTransform':None,
        'scale':10
      })
      '''
      img_10km=reduceResolution_10km(img_m,p1,10)

    else:
      print('should input these crop names: soyb/rice/sugc/cott/whea/rape/maiz')
  '''
  spam2010_table=pd.read_csv('/content/drive/MyDrive/global_crop_dataset/SPAM2010_crop_area_table-CHN.csv')
  adm_list=spam2010_table['ADM1_NAME'].values.tolist()

  GEO_count=-1
  for GEO_name in adm_list:
    GEO_count+=1
    #spam2010_table=pd.read_csv('/content/drive/MyDrive/spam2010_suit-africa/spam2010_suit_'+str(GEO_count))

    geo_adm=chn_adm1
    geo_bound=geo_adm.filter(ee.Filter.eq("ADM1_NAME",GEO_name))

    task1=ee.batch.Export.image.toAsset(**{
      'image':img_10km,
      'description':crop_name+'_'+str(year)+'_'+str(GEO_count),
      'assetId':"projects/ee-global-crop/assets/CDL_CHN_multiyear_base_layer_10km/"+crop_name+'_'+str(year)+'_'+str(GEO_count),
      'region':geo_bound.geometry(),
      'scale':10000,
    })
    task1.start()
  '''
  for id in chn_fishnet_Id_list:

    geo_adm=fishnet
    geo_bound=geo_adm.filter(ee.Filter.eq("Id",id))

    task1=ee.batch.Export.image.toAsset(**{
      'image':img_10km,
      'description':crop_name+'_'+str(year)+'_'+str(id)+'_v2',
      'assetId':"projects/ee-china-crop/assets/CDL_CHN_multiyear_base_layer_10km_1/"+crop_name+'_'+str(year)+'_'+str(id)+'_v2',
      'region':geo_bound.geometry(),
      'scale':10000,
    })
    task1.start()

  '''
  task1=ee.batch.Export.image.toDrive(**{
      'image':img_m,
      'description':"SPAM_GEO_"+str(GEO_count)+'_'+str(2010),
      'folder':"Crop_map_CHN_SPAM_refer",
      'region':chn_adm1.geometry(),
      'scale':10000,
    })
  task1.start()
  '''

In [ ]:
def get_china_spam_2010_map():
  for crop_china_name in china_crop_list:
    #crop_name=china_crop_list[id]
    id=china_crop_list.index(crop_china_name)
    spam_list=spam_china_crop_list[id]
    multi_year_bool=multi_year_data_training[id]

    spam_list_len=len(spam_list)
    for SPAM_shortName in spam_list:
      if spam_list_len==1:
        spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2010/spam2010V2r0_global_H_'+SPAM_shortName.upper()+'_A')
        spam_a=spam_a.updateMask(spam_a.gt(0)).unmask(0)
      else:
        if SPAM_shortName==spam_list[0]:
          spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2010/spam2010V2r0_global_H_'+SPAM_shortName.upper()+'_A')
          spam_a=spam_a.updateMask(spam_a.gt(0)).unmask(0)
        else:
          spam_a0=ee.Image('projects/ee-global-crop/assets/SPAM2010/spam2010V2r0_global_H_'+SPAM_shortName.upper()+'_A')
          spam_a0=spam_a0.updateMask(spam_a0.gt(0)).unmask(0)
          spam_a=spam_a.add(spam_a0)

    if id==0:
      spam_f=spam_a.rename(str(id))
    else:
      spam_f=spam_f.addBands(spam_a.rename(str(id)))

  return spam_f

In [ ]:
def export_SPAM2010_crop_area_table_CHN(f):
  final_f=ee.Feature(f.geometry())
  final_f=final_f.set('ADM1_NAME',f.get('ADM1_NAME')).set('ADM1_CODE',f.get('ADM1_CODE'))

  spam_china=get_china_spam_2010_map()
  for crop_china_name in china_crop_list:
    id=china_crop_list.index(crop_china_name)
    spam_area_a=spam_china.select(str(id))

    crop_sum_a=spam_area_a.rename(str(id)+'_area').reduceRegion(**{
      'geometry':f.geometry(),
      'reducer':ee.Reducer.sum(),
      'scale':10000,
      'tileScale':8
    }).get(str(id)+'_area')

    final_f=final_f.set(str(id)+'_a',crop_sum_a)
  return final_f

exp=chn_adm1.map(export_SPAM2010_crop_area_table_CHN)
task=ee.batch.Export.table.toDrive(**{
  'collection':exp,
  'description':'SPAM2010_crop_area_table-CHN',
  'folder':'global_crop_dataset',
  'fileFormat':'csv',
})
#collection, description, folder, fileNamePrefix, fileFormat, selectors, maxVertices
task.start()

In [ ]:
def export_base_map_by_adm1_CHN():
  spam2010_table=pd.read_csv('/content/drive/MyDrive/global_crop_dataset/SPAM2010_crop_area_table-CHN.csv')
  adm_list=spam2010_table['ADM1_NAME'].values.tolist()

  spam_china=get_china_spam_2010_map()

  GEO_count=-1
  for GEO_name in adm_list:
    GEO_count+=1
    #spam2010_table=pd.read_csv('/content/drive/MyDrive/spam2010_suit-africa/spam2010_suit_'+str(GEO_count))

    geo_adm=chn_adm1
    geo_bound=geo_adm.filter(ee.Filter.eq("ADM1_NAME",GEO_name))

    task1=ee.batch.Export.image.toDrive(**{
      'image':spam_china.clip(geo_bound),
      'description':"SPAM_GEO_"+str(GEO_count)+'_'+str(2010),
      'folder':"Crop_map_CHN_SPAM_refer",
      'region':geo_bound.geometry(),
      'scale':10000,
    })
    task1.start()

In [ ]:
export_base_map_by_adm1_CHN()

# **RF predict**

In [ ]:
#not distinguish add or reduce //add climate soil pop field agri variables
def merge_indicator_band_as_one_image_RF(crop_name,year):
  id=crop_name_list.index(crop_name)

  def ImgtoFloat(img):
    return img.toFloat()
  '''
  const_indicator=ee.ImageCollection('projects/ee-global-crop/assets/Crop_prob_USA_const').map(lambda img: ImgtoFloat(img)).mosaic()
  test_img=ee.ImageCollection('projects/ee-global-crop/assets/Crop_prob_USA_const').first()
  p1=test_img.projection()
  const_indicator=const_indicator.setDefaultProjection(**{
    'crs': p1,
    'crsTransform':None,
    'scale':10000
  })
  change_indicator=ee.ImageCollection('projects/ee-global-crop/assets/Crop_prob_USA_change_0822').filter(ee.Filter.stringContains('system:index',str(year))).map(lambda img: ImgtoFloat(img)).mosaic()
  change_indicator=change_indicator.setDefaultProjection(**{
    'crs': p1,
    'crsTransform':None,
    'scale':10000
  })
  '''
  era5_month=ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR").filter(ee.Filter.calendarRange(year, year, 'year'))
  era5_daily=ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR").filter(ee.Filter.calendarRange(year, year, 'year'))
  p_era5=era5_month.first().projection()
  p_era5_daily=era5_daily.first().projection()

  prec=era5_month.select('total_precipitation_sum').sum().multiply(1000).rename('prec')
  temp=era5_month.select('temperature_2m').mean().subtract(273.15).rename('temp')
  radi_down=era5_month.select('surface_solar_radiation_downwards_sum').sum().rename('radi_down')
  evap_veg=era5_month.select('evaporation_from_vegetation_transpiration_sum').sum().rename('evap_veg')

  '''
  SPI_c=ee.ImageCollection([])
  SPEI_c=ee.ImageCollection([])
  for month in range(1,13,1):
    num=year-1959
    SPI=ee.Image('projects/ee-tfp-climate-change/assets/ERA5_SPI-6'+'Monthly_SPI-6_from_ERA5_195901-20221_spi_'+str(num*12+month))
    SPEI=ee.Image('projects/ee-tfp-climate-change/assets/ERA5_SPEI-6'+'Monthly_SPEI-6_from_ERA5_195901-20221_spei_'+str(num*12+month))
    SPI_c=SPI_c.merge(ee.ImageCollection(SPI))
    SPEI_c=SPEI_c.merge(ee.ImageCollection(SPEI))
  #SPI=ee.ImageCollection('projects/ee-tfp-climate-change/assets/ERA5_SPI-6').filter(ee.Filter.calendarRange(year, year, 'year'))
  #SPEI=ee.ImageCollection('projects/ee-tfp-climate-change/assets/ERA5_SPEI-6').filter(ee.Filter.calendarRange(year, year, 'year'))
  '''

  def getGDD(img):
    max=img.select('temperature_2m_max').subtract(273.15).rename('temp')
    min=img.select('temperature_2m_min').subtract(273.15).rename('temp')
    cap=ee.ImageCollection([max,ee.Image(30).rename('temp')]).min()
    base=ee.ImageCollection([min,ee.Image(10).rename('temp')]).max()
    gdd_0=((cap.add(base)).divide(2)).subtract(10).rename('temp')
    gdd=ee.ImageCollection([gdd_0,ee.Image(0).rename('temp')]).max().rename('gdd')
    return gdd
  gdd_daily=era5_daily.map(lambda img: getGDD(img))
  gdd=gdd_daily.select('gdd').sum().rename('gdd')

  prec=prec.setDefaultProjection(**{
      'crs': p_era5,
      'crsTransform':None,
      'scale':11132
    })
  temp=temp.setDefaultProjection(**{
      'crs': p_era5,
      'crsTransform':None,
      'scale':11132
    })
  radi_down=radi_down.setDefaultProjection(**{
      'crs': p_era5,
      'crsTransform':None,
      'scale':11132
    })
  evap_veg=evap_veg.setDefaultProjection(**{
      'crs': p_era5,
      'crsTransform':None,
      'scale':11132
    })

  prec_10km = prec.reduceResolution(**{
      'reducer': ee.Reducer.mean(),
      'bestEffort':True,
      'maxPixels':60000
    }).reproject(p_era5,None,10000).rename('prec')

  temp_10km = temp.reduceResolution(**{
      'reducer': ee.Reducer.mean(),
      'bestEffort':True,
      'maxPixels':60000
    }).reproject(p_era5,None,10000).rename('temp')
  radi_down_10km = radi_down.reduceResolution(**{
      'reducer': ee.Reducer.mean(),
      'bestEffort':True,
      'maxPixels':60000
    }).reproject(p_era5,None,10000).rename('radi_down')
  evap_veg_10km = evap_veg.reduceResolution(**{
      'reducer': ee.Reducer.mean(),
      'bestEffort':True,
      'maxPixels':60000
    }).reproject(p_era5,None,10000).rename('evap_veg')

  gdd=gdd.setDefaultProjection(**{
      'crs': p_era5,
      'crsTransform':None,
      'scale':11132
    })
  gdd_10km = gdd.reduceResolution(**{
      'reducer': ee.Reducer.mean(),
      'bestEffort':True,
      'maxPixels':60000
    }).reproject(p_era5,None,10000).rename('gdd')


  soil_texture_m_10km=ee.Image("projects/ee-global-crop/assets/envr_indicator/"+"soil_texture_m_10km")
  soil_water_m_10km=ee.Image("projects/ee-global-crop/assets/envr_indicator/"+"soil_water_m_10km")
  soil_clay_m_10km=ee.Image("projects/ee-global-crop/assets/envr_indicator/"+"soil_clay_m_10km")
  soil_orgnC_m_10km=ee.Image("projects/ee-global-crop/assets/envr_indicator/"+"soil_orgnC_m_10km")
  soil_pH_m_10km=ee.Image("projects/ee-global-crop/assets/envr_indicator/"+"soil_pH_m_10km")
  soil_sand_m_10km=ee.Image("projects/ee-global-crop/assets/envr_indicator/"+"soil_sand_m_10km")
  dem_10km=ee.Image("projects/ee-global-crop/assets/envr_indicator/"+"dem_10km_full")
  slope_10km=ee.Image("projects/ee-global-crop/assets/envr_indicator/"+"slope_10km_full")
  field_size_10km=ee.Image("projects/ee-global-crop/assets/envr_indicator/"+"field_size_10km")

  #pop rurc hyde
  if year<1970 and year>=1960:
    year_hyde='1960AD'
    pop=ee.ImageCollection('projects/ee-hyde/assets/rurc').filter(ee.Filter.stringContains('system:index',year_hyde)).first().rename('rurc')
  elif year<1980 and year>=1970:
    year_hyde='1970AD'
    pop=ee.ImageCollection('projects/ee-hyde/assets/rurc').filter(ee.Filter.stringContains('system:index',year_hyde)).first().rename('rurc')
  elif year<1990 and year>=1980:
    year_hyde='1980AD'
    pop=ee.ImageCollection('projects/ee-hyde/assets/rurc').filter(ee.Filter.stringContains('system:index',year_hyde)).first().rename('rurc')
  elif year<2000 and year>=1990:
    year_hyde='1990AD'
    pop=ee.ImageCollection('projects/ee-hyde/assets/rurc').filter(ee.Filter.stringContains('system:index',year_hyde)).first().rename('rurc')
  elif year<=2017 and year>=2000:
    year_hyde=str(year)+'AD'
    pop=ee.ImageCollection('projects/ee-hyde/assets/rurc').filter(ee.Filter.stringContains('system:index',year_hyde)).first().rename('rurc')
  elif year>2017:
    pop=ee.ImageCollection('projects/ee-hyde/assets/rurc').filter(ee.Filter.stringContains('system:index','2017AD')).first().rename('rurc')

  p_hyde=ee.ImageCollection('projects/ee-hyde/assets/rurc').filter(ee.Filter.stringContains('system:index','2017AD')).first().projection()

  pop=pop.setDefaultProjection(**{
      'crs': p_hyde,
      'crsTransform':None,
      'scale':9276.62
    })
  pop_10km = pop.reduceResolution(**{
      'reducer': ee.Reducer.mean(),
      'bestEffort':True,
      'maxPixels':60000
    }).reproject(p_hyde,None,10000).rename('rurc')

  cropland=get_cropland_layer(year).rename('cropland')

  #farming system irrigation or?
  if year>=1998:
    if year<=2002:
      system_share=ee.Image('projects/ee-global-crop/assets/cropland_system_share_SPAMphys/cropland_system_share_SPAMphys_'+'2000')
    elif year>2002 and year<2008:
      system_share=ee.Image('projects/ee-global-crop/assets/cropland_system_share_SPAMphys/cropland_system_share_SPAMphys_'+'2005')
    else:
      system_share=ee.Image('projects/ee-global-crop/assets/cropland_system_share_SPAMphys/cropland_system_share_SPAMphys_'+'2010')

    system_share_i=system_share.select('I').rename('system_share_i')
    system_share_r=system_share.select('R').rename('system_share_r')
  else:#year<=1997
    if year>1992:
      year_irri='1995'
      irri_area=ee.ImageCollection('projects/ee-global-crop/assets/IrrigationArea_multiyear').filter(ee.Filter.stringContains('system:index',year_irri)).first()
    elif year>=1988 and year<=1992:
      year_irri='1990'
      irri_area=ee.ImageCollection('projects/ee-global-crop/assets/IrrigationArea_multiyear').filter(ee.Filter.stringContains('system:index',year_irri)).first()
    elif year>1982 and year<1988:
      year_irri='1985'
      irri_area=ee.ImageCollection('projects/ee-global-crop/assets/IrrigationArea_multiyear').filter(ee.Filter.stringContains('system:index',year_irri)).first()
    elif year>=1975 and year<=1982:
      year_irri='1980'
      irri_area=ee.ImageCollection('projects/ee-global-crop/assets/IrrigationArea_multiyear').filter(ee.Filter.stringContains('system:index',year_irri)).first()
    elif year>1972 and year<1975:
      year_irri='1970'
      irri_area=ee.ImageCollection('projects/ee-global-crop/assets/IrrigationArea_multiyear').filter(ee.Filter.stringContains('system:index',year_irri)).first()
    elif year>=1965 and year<=1972:
      year_irri='1970'
      irri_area=ee.ImageCollection('projects/ee-global-crop/assets/IrrigationArea_multiyear').filter(ee.Filter.stringContains('system:index',year_irri)).first()
    elif year>1962 and year<1965:
      year_irri='1960'
      irri_area=ee.ImageCollection('projects/ee-global-crop/assets/IrrigationArea_multiyear').filter(ee.Filter.stringContains('system:index',year_irri)).first()
    else:
      year_irri='1960'
      irri_area=ee.ImageCollection('projects/ee-global-crop/assets/IrrigationArea_multiyear').filter(ee.Filter.stringContains('system:index',year_irri)).first()

    pixelArea_10km=ee.Image('projects/ee-global-crop/assets/pixelArea_global')
    cropland_area=cropland.multiply(pixelArea_10km).divide(10000)
    cropland_area_match_irri=cropland_area.where(cropland_area.lt(irri_area),irri_area)
    system_share_i=irri_area.divide(cropland_area_match_irri).rename('system_share_i')
    system_share_r=ee.Image(1).subtract(system_share_i).rename('system_share_r')


  climate_indicator=temp_10km.addBands(prec_10km).addBands(radi_down_10km).addBands(evap_veg_10km).addBands(gdd_10km)#.addBands(SPI).addBands(SPEI)
  farm_indicator=soil_texture_m_10km.addBands(soil_water_m_10km).addBands(soil_clay_m_10km).addBands(soil_orgnC_m_10km).addBands(soil_pH_m_10km).addBands(soil_sand_m_10km).addBands(dem_10km).addBands(slope_10km).addBands(cropland).addBands(pop_10km).addBands(field_size_10km).addBands(system_share_i).addBands(system_share_r)


  suit_i=produce_suit_layer(id+1,year,'i').rename('suit_i')
  suit_h=produce_suit_layer(id+1,year,'h').rename('suit_h')
  suit_l=produce_suit_layer(id+1,year,'l').rename('suit_l')

  potYield_i=produce_PotYield_layer(id+1,year,'i').rename('yield_i')
  potYield_h=produce_PotYield_layer(id+1,year,'h').rename('yield_h')
  potYield_l=produce_PotYield_layer(id+1,year,'l').rename('yield_l')


  GAEZ_indicator=suit_i.addBands(suit_h).addBands(suit_l).addBands(potYield_i).addBands(potYield_h).addBands(potYield_l)

  pixelLocation=ee.Image('projects/ee-global-crop/assets/pixelLocation_global')
  '''
  hist=const_indicator.select('cropHist_'+str(id)).rename('hist')
  crop_sturc=ee.Image("projects/ee-global-crop/assets/crop_stucture_system_share_SPAMphys/"+crop_name+'_'+str(2010)).select('b1')
  crop_sturc=crop_sturc.where(crop_sturc.lt(0),0)
  crop_sturc=crop_sturc.where(crop_sturc.gt(1),1)
  crop_sturc=crop_sturc.rename('crop_struc')

  reduce_crop_prop=change_indicator.select('re_'+str(year)).rename('reduce_crop_prop')
  add_crop_prop=change_indicator.select('add_'+str(year)).rename('add_crop_prop')
  '''

  indicator=climate_indicator.addBands(farm_indicator).addBands(GAEZ_indicator).addBands(pixelLocation)
  #indicator=climate_indicator.addBands(farm_indicator).addBands(pixelLocation)

  return indicator

# **USA_export_prob_layer**

In [ ]:
#sample with Targit area as value
def CDL_stratifiedSample(year):
  for crop_name in crop_name_list:
    id=crop_name_list.index(crop_name)
    USDA_name=usda_name_list[id]

    if USDA_name=='BEANS':
      continue
    if USDA_name !='':
      refer_map=ee.Image("projects/ee-global-crop/assets/CDL_10km_area/CDL_10km_area_p_"+str(year)+'_'+USDA_name)
      refer_map=refer_map.rename('crop_area_prop')
      #refer_map=refer_map.divide(pixelArea_10km).rename('crop_area_prop')#transfer abosulute value into prop
      refer_map_0=refer_map.neq(0)
      sample_all=refer_map_0.stratifiedSample(**{
        'numPoints':10000,
        'classBand':'crop_area_prop',
        'scale':10000,
        'tileScale':16,
        'geometries':True,
      });
      task1=ee.batch.Export.table.toAsset(**{
        'collection':sample_all,
        'description':crop_name+'_'+str(year),
        'assetId':'projects/ee-global-crop/assets/CDL_sample_3/'+crop_name+'_'+str(year),
      });
      task1.start()

def train_RF_and_get_Predict_Result(sampleData,input_img):
  withRandom = sampleData.randomColumn('random');

  split = 0.7;
  training = withRandom.filter(ee.Filter.lt('random', split));
  testing = withRandom.filter(ee.Filter.gte('random', split));

  inputProperties = input_img.bandNames().remove('crop_area_prop_1')#sampleData.first().propertyNames().remove('change_area').remove('system:index');
  classProperties = 'crop_area_prop_1';

  #build classifier
  classifier = ee.Classifier.smileRandomForest(100).setOutputMode('REGRESSION').train(**{
    'features': training,
    'classProperty': classProperties,
    'inputProperties' : inputProperties,
  });

  input_img=input_img.select(inputProperties)
  res=input_img.classify(classifier)
  return res

def export_indicator_sample_collection_RF(train_year,predict_year):
  crop_type_list=crop_type_replace['crop_id'].tolist()
  for crop_id in crop_type_list:
    crop_type_selected=crop_type_replace.loc[crop_type_replace['crop_id'] == crop_id]
    SPAM_shortName=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("SPAM_shortName")])
    crop_name=SPAM_shortName
    id=crop_name_list.index(SPAM_shortName)
    USDA_name=usda_name_list[id]
    if USDA_name=='BEANS':
      continue
    if USDA_name !='':
      train_map=ee.Image("projects/ee-global-crop/assets/CDL_10km_area/CDL_10km_area_p_"+str(train_year)+'_'+USDA_name)
      #train_map=pixelArea_10km.multiply(train_map).divide(10000)

      train_map=train_map.rename('crop_area_prop_1')
      #train_map=train_map.divide(pixelArea_10km).rename('crop_area_prop')#transfer abosulute value into prop

      #test_map=ee.Image("projects/ee-global-crop/assets/CDL_10km_area/CDL_10km_area_p_"+str(predict_year)+'_'+USDA_name)
      ##test_map=pixelArea_10km.multiply(test_map).divide(10000)

      #test_map=test_map.rename('crop_area_prop_1')
      ##test_map=test_map.divide(pixelArea_10km).rename('crop_area_prop')#transfer abosulute value into prop

      indicator_train=merge_indicator_band_as_one_image_RF(crop_name,train_year).addBands(train_map)
      indicator_test=merge_indicator_band_as_one_image_RF(crop_name,predict_year)#.addBands(test_map)
      sample=ee.FeatureCollection('projects/ee-global-crop/assets/CDL_sample_3/'+crop_name+'_'+str(train_year)).filter(ee.Filter.eq('crop_area_prop',1))
      indicator_f=indicator_train.sampleRegions(**{
        'collection':sample,
        #properties:,
        'scale':10000,
        #projection:,
        #'tileScale':16,
        #geometries:
      })
      print(crop_name+':')
      print(indicator_f.first().propertyNames().getInfo())
      res=train_RF_and_get_Predict_Result(indicator_f,indicator_test)

    if crop_id==1:
      exp=res.rename(crop_name)
    else:
      exp=exp.addBands(res.rename(crop_name))

  task1=ee.batch.Export.image.toAsset(**{
    'image':exp,
    'description':"prob_test_"+"train_"+str(train_year)+"_predict_"+str(predict_year)+'_3',
    'assetId':"projects/ee-global-crop/assets/Crop_prob_USA_RF4/"+"prob_test_"+"train_"+str(train_year)+"_predict_"+str(predict_year)+'_3',
    'region':usa_adm0.geometry(),
    'scale':10000,
  })
  task1.start()


In [ ]:
for year in range(2008,2023):
  if year==2010:
    continue
  export_indicator_sample_collection_RF(train_year=2010,predict_year=year)

In [ ]:
for year in range(2008,2023):
  if year==2010:
    continue
  CDL_stratifiedSample(year)

In [ ]:
CDL_stratifiedSample(2010)

In [ ]:
def train_RF_and_get_accuracy_accessment(sampleData,input_img):
  withRandom = sampleData.randomColumn('random');

  split = 0.7;
  training = withRandom.filter(ee.Filter.lt('random', split));
  testing = withRandom.filter(ee.Filter.gte('random', split));

  inputProperties = input_img.bandNames().remove('crop_area_prop_1')#sampleData.first().propertyNames().remove('change_area').remove('system:index');
  classProperties = 'crop_area_prop_1';

  #build classifier
  classifier = ee.Classifier.smileRandomForest(100).setOutputMode('REGRESSION').train(**{
    'features': training,
    'classProperty': classProperties,
    'inputProperties' : inputProperties,
  });

  res_test=testing.classify(classifier)


In [ ]:
def export_indicator_sample_collection_RF_multiYear(train_year_list,predict_year):
  crop_type_list=crop_type_replace['crop_id'].tolist()
  for crop_id in crop_type_list:
    crop_type_selected=crop_type_replace.loc[crop_type_replace['crop_id'] == crop_id]
    SPAM_shortName=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("SPAM_shortName")])
    crop_name=SPAM_shortName
    id=crop_name_list.index(SPAM_shortName)
    USDA_name=usda_name_list[id]
    if USDA_name=='BEANS':
      continue
    if USDA_name !='':
      for train_year in train_year_list:

        train_map=ee.Image("projects/ee-global-crop/assets/CDL_10km_area/CDL_10km_area_p_"+str(train_year)+'_'+USDA_name)
        #train_map=pixelArea_10km.multiply(train_map).divide(10000)

        train_map=train_map.rename('crop_area_prop_1')
        #train_map=train_map.divide(pixelArea_10km).rename('crop_area_prop')#transfer abosulute value into prop

        #test_map=ee.Image("projects/ee-global-crop/assets/CDL_10km_area/CDL_10km_area_p_"+str(predict_year)+'_'+USDA_name)
        ##test_map=pixelArea_10km.multiply(test_map).divide(10000)

        #test_map=test_map.rename('crop_area_prop_1')
        ##test_map=test_map.divide(pixelArea_10km).rename('crop_area_prop')#transfer abosulute value into prop

        indicator_train=merge_indicator_band_as_one_image_RF(crop_name,train_year).addBands(train_map)
        indicator_test=merge_indicator_band_as_one_image_RF(crop_name,predict_year)#.addBands(test_map)
        sample=ee.FeatureCollection('projects/ee-global-crop/assets/CDL_sample_3/'+crop_name+'_'+str(train_year)).filter(ee.Filter.eq('crop_area_prop',1))
        indicator_f0=indicator_train.sampleRegions(**{
          'collection':sample,
          #properties:,
          'scale':10000,
          #projection:,
          #'tileScale':16,
          #geometries:
        })
        if train_year==train_year_list[0]:
          indicator_f=indicator_f0
        else:
          indicator_f=indicator_f.merge(indicator_f0)

      print(crop_name+':')
      print(indicator_f.first().propertyNames().getInfo())
      res=train_RF_and_get_Predict_Result(indicator_f,indicator_test)

    if crop_id==1:
      exp=res.rename(crop_name)
    else:
      exp=exp.addBands(res.rename(crop_name))

  task1=ee.batch.Export.image.toAsset(**{
    'image':exp,
    'description':"prob_test_"+"train_"+'multiYear'+"_predict_"+str(predict_year),
    'assetId':"projects/ee-global-crop/assets/Crop_prob_USA_RF5_multiYear/"+"prob_test_"+"train_"+'multiYear'+"_predict_"+str(predict_year),
    'region':usa_adm0.geometry(),
    'scale':10000,
  })
  task1.start()

# **AF_export_prob_layer**

In [ ]:
#sample with Targit area as value
def CDL_stratifiedSample(year):
  for crop_name in crop_name_list:
    SPAM_shortName=crop_name
    if year==2010:
      spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2010/spam2010V2r0_global_H_'+SPAM_shortName.upper()+'_A').clip(africa)
      spam_a=spam_a.updateMask(spam_a.gte(0)).unmask(0)
    elif year==2005:
      spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2005/SPAM2005V3r2_global_H_T'+'A_'+SPAM_shortName.upper()+'_A').clip(africa)
      spam_a=spam_a.updateMask(spam_a.gte(0)).unmask(0)
    refer_map=spam_a.divide(pixelArea_10km).multiply(10000)
    refer_map=refer_map.rename('crop_area_prop').updateMask(refer_map)

    #refer_map=refer_map.divide(pixelArea_10km).rename('crop_area_prop')#transfer abosulute value into prop
    refer_map_0=refer_map.neq(0)
    sample_all=refer_map_0.stratifiedSample(**{
      'numPoints':10000,
      'classBand':'crop_area_prop',
      'scale':10000,
      'tileScale':16,
      'geometries':True,
    });
    task1=ee.batch.Export.table.toAsset(**{
      'collection':sample_all,
      'description':crop_name+'_'+str(year),
      'assetId':'projects/ee-global-crop/assets/Crop_sample_AF/'+crop_name+'_'+str(year),
    });
    task1.start()

def train_RF_and_get_Predict_Result(sampleData,input_img):
  withRandom = sampleData.randomColumn('random');

  split = 0.7;
  training = withRandom.filter(ee.Filter.lt('random', split));
  testing = withRandom.filter(ee.Filter.gte('random', split));

  inputProperties = input_img.bandNames().remove('crop_area_prop_1')#sampleData.first().propertyNames().remove('change_area').remove('system:index');
  classProperties = 'crop_area_prop_1';

  #build classifier
  classifier = ee.Classifier.smileRandomForest(100).setOutputMode('REGRESSION').train(**{
    'features': training,
    'classProperty': classProperties,
    'inputProperties' : inputProperties,
  });

  input_img=input_img.select(inputProperties)
  res=input_img.classify(classifier)

  return res

def export_indicator_sample_collection_RF(train_year,predict_year):
  crop_type_list=crop_type_replace['crop_id'].tolist()
  for crop_id in crop_type_list:
    crop_type_selected=crop_type_replace.loc[crop_type_replace['crop_id'] == crop_id]
    SPAM_shortName=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("SPAM_shortName")])
    crop_name=SPAM_shortName

    if train_year==2010:
      spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2010/spam2010V2r0_global_H_'+SPAM_shortName.upper()+'_A').clip(africa)
      spam_a=spam_a.updateMask(spam_a.gte(0)).unmask(0)
    elif train_year==2005:
      spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2005/SPAM2005V3r2_global_H_T'+'A_'+SPAM_shortName.upper()+'_A').clip(africa)
      spam_a=spam_a.updateMask(spam_a.gte(0)).unmask(0)

    train_map=spam_a.divide(pixelArea_10km).multiply(10000)
    #train_map=pixelArea_10km.multiply(train_map).divide(10000)

    train_map=train_map.rename('crop_area_prop_1')

    indicator_train=merge_indicator_band_as_one_image_RF(crop_name,train_year).addBands(train_map)
    indicator_test=merge_indicator_band_as_one_image_RF(crop_name,predict_year)#.addBands(test_map)
    sample=ee.FeatureCollection('projects/ee-global-crop/assets/Crop_sample_AF/'+crop_name+'_'+str(train_year)).filter(ee.Filter.eq('crop_area_prop',1))
    indicator_f=indicator_train.sampleRegions(**{
      'collection':sample,
      #properties:,
      'scale':10000,
      #projection:,
      #'tileScale':16,
      #geometries:
    })
    print(crop_name+':')
    print(indicator_f.first().propertyNames().getInfo())
    res=train_RF_and_get_Predict_Result(indicator_f,indicator_test)

    if crop_id==1:
      exp=res.rename(crop_name)
    else:
      exp=exp.addBands(res.rename(crop_name))

  task1=ee.batch.Export.image.toAsset(**{
    'image':exp,
    'description':"prob_test_"+"train_"+str(train_year)+"_predict_"+str(predict_year),
    'assetId':"projects/ee-global-crop/assets/Crop_prob_AF_RF/"+"prob_test_"+"train_"+str(train_year)+"_predict_"+str(predict_year),
    'region':africa.geometry(),
    'scale':10000,
  })
  task1.start()

# **CHN_export_prob_layer**

In [ ]:
#sample with Targit area as value
def CDL_stratifiedSample(year):
  for crop_name in spam_china_crop_list_1:
    SPAM_shortName=crop_name
    if year==2010:
      spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2010/spam2010V2r0_global_H_'+SPAM_shortName.upper()+'_A').clip(chn_adm1)
      spam_a=spam_a.updateMask(spam_a.gte(0)).unmask(0)
    elif year==2005:
      spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2005/SPAM2005V3r2_global_H_T'+'A_'+SPAM_shortName.upper()+'_A').clip(chn_adm1)
      spam_a=spam_a.updateMask(spam_a.gte(0)).unmask(0)
    refer_map=spam_a.divide(pixelArea_10km).multiply(10000)
    refer_map=refer_map.rename('crop_area_prop').updateMask(refer_map)

    #refer_map=refer_map.divide(pixelArea_10km).rename('crop_area_prop')#transfer abosulute value into prop
    refer_map_0=refer_map.neq(0)
    sample_all=refer_map_0.stratifiedSample(**{
      'numPoints':10000,
      'region':chn_adm1,
      'classBand':'crop_area_prop',
      'scale':10000,
      'tileScale':16,
      'geometries':True,
    });
    task1=ee.batch.Export.table.toAsset(**{
      'collection':sample_all,
      'description':crop_name+'_'+str(year),
      'assetId':'projects/ee-global-crop-1/assets/Crop_sample_CHN/'+crop_name+'_'+str(year),
    });
    task1.start()

def train_RF_and_get_Predict_Result(sampleData,input_img):
  withRandom = sampleData.randomColumn('random');

  split = 0.7;
  training = withRandom.filter(ee.Filter.lt('random', split));
  testing = withRandom.filter(ee.Filter.gte('random', split));

  inputProperties = input_img.bandNames().remove('crop_area_prop_1')#sampleData.first().propertyNames().remove('change_area').remove('system:index');
  classProperties = 'crop_area_prop_1';

  #build classifier
  classifier = ee.Classifier.smileRandomForest(100).setOutputMode('REGRESSION').train(**{
    'features': training,
    'classProperty': classProperties,
    'inputProperties' : inputProperties,
  });

  input_img=input_img.select(inputProperties)
  res=input_img.classify(classifier)

  return res

def export_indicator_sample_collection_RF(train_year,predict_year):
  crop_id=0
  for crop_name in spam_china_crop_list_1:
    if crop_name=='pige'or crop_name=='yams'or crop_name=='chic' or crop_name=='cowp':
      continue
    crop_id+=1
    SPAM_shortName=crop_name

    if train_year==2010:
      spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2010/spam2010V2r0_global_H_'+SPAM_shortName.upper()+'_A').clip(chn_adm1)
      spam_a=spam_a.updateMask(spam_a.gte(0)).unmask(0)
    elif train_year==2005:
      spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2005/SPAM2005V3r2_global_H_T'+'A_'+SPAM_shortName.upper()+'_A').clip(chn_adm1)
      spam_a=spam_a.updateMask(spam_a.gte(0)).unmask(0)

    train_map=spam_a.divide(pixelArea_10km).multiply(10000)
    #train_map=pixelArea_10km.multiply(train_map).divide(10000)

    train_map=train_map.rename('crop_area_prop_1')

    indicator_train=merge_indicator_band_as_one_image_RF(crop_name,train_year).addBands(train_map)
    indicator_test=merge_indicator_band_as_one_image_RF(crop_name,predict_year)#.addBands(test_map)
    sample=ee.FeatureCollection('projects/ee-global-crop-1/assets/Crop_sample_CHN/'+crop_name+'_'+str(train_year)).filter(ee.Filter.eq('crop_area_prop',1))
    indicator_f=indicator_train.sampleRegions(**{
      'collection':sample,
      #properties:,
      'scale':10000,
      #projection:,
      #'tileScale':16,
      #geometries:
    })
    print(crop_name+':')
    print(indicator_f.first().propertyNames().getInfo())
    res=train_RF_and_get_Predict_Result(indicator_f,indicator_test)

    if crop_id==1:
      exp=res.rename(crop_name)
    else:
      exp=exp.addBands(res.rename(crop_name))

  task1=ee.batch.Export.image.toAsset(**{
    'image':exp,
    'description':"prob_test_"+"train_"+str(train_year)+"_predict_"+str(predict_year),
    'assetId':"projects/ee-global-crop-1/assets/Crop_prob_CHN_RF/"+"prob_test_"+"train_"+str(train_year)+"_predict_"+str(predict_year),
    'region':chn_adm1.geometry(),
    'scale':10000,
  })
  task1.start()

In [ ]:
for year in range(1980,2023,1):
  for crop_china_name in china_crop_list:
    #crop_name=china_crop_list[id]
    id=china_crop_list.index(crop_china_name)
    spam_list=spam_china_crop_list[id]
    multi_year_bool=multi_year_data_training[id]

    spam_list_len=len(spam_list)
    if spam_list_len!=1:
      continue

    SPAM_shortName=spam_list[0]

    if multi_year_bool==0:
      continue

    train_year_list=train_year_list_list[id]
    crop_name=SPAM_shortName

In [ ]:
def CDL_stratifiedSample_multiYear():
  for crop_china_name in china_crop_list:
    #crop_name=china_crop_list[id]
    id=china_crop_list.index(crop_china_name)
    spam_list=spam_china_crop_list[id]
    multi_year_bool=multi_year_data_training[id]

    spam_list_len=len(spam_list)
    if spam_list_len!=1:
      continue

    SPAM_shortName=spam_list[0]
    crop_name=SPAM_shortName

    if crop_name!='soyb':
      continue

    if multi_year_bool==0:
      continue

    train_year_list=train_year_list_list[id]

    for year in train_year_list:
      #spam_a=ee.ImageCollection('projects/ee-global-crop/assets/CDL_CHN_multiyear_base_layer_10km_1').filter(ee.Filter.stringContains('system:index',crop_name)).filter(ee.Filter.stringContains('system:index',str(year))).qualityMosaic('b1').clip(chn_adm1)
      spam_a=ee.Image("projects/ee-global-crop/assets/CDL_CHN_multiyear_base_layer_10km/"+crop_name+'_'+str(year))
      '''
      task1=ee.batch.Export.image.toAsset(**{
        'image':spam_a,
        'description':crop_name+'_'+str(year),
        'assetId':"projects/ee-global-crop/assets/CDL_CHN_multiyear_base_layer_10km/"+crop_name+'_'+str(year),
        'region':chn_adm1.geometry(),
        'scale':10000,
      })
      task1.start()
      '''
      refer_map=spam_a#.divide(pixelArea_10km).multiply(10000)
      #train_map=pixelArea_10km.multiply(train_map).divide(10000)

      refer_map=refer_map.rename('crop_area_prop')


      refer_map_0=refer_map.neq(0)
      sample_all=refer_map_0.stratifiedSample(**{
        'numPoints':10000,
        'region':spam_a.geometry(),
        'classBand':'crop_area_prop',
        'scale':10000,
        'tileScale':16,
        'geometries':True,
      });
      #print(sample_all.first().propertyNames())
      task1=ee.batch.Export.table.toAsset(**{
        'collection':sample_all,
        'description':crop_name+'_'+str(year),
        'assetId':'projects/ee-global-crop-1/assets/Crop_sample_CHN_multiYear_1/'+crop_name+'_'+str(year),
      });
      task1.start()

In [ ]:
CDL_stratifiedSample_multiYear()

In [ ]:
def export_indicator_sample_collection_RF_multiYear(predict_year):
  crop_id=0
  for crop_china_name in china_crop_list:
    #crop_name=china_crop_list[id]
    id=china_crop_list.index(crop_china_name)
    spam_list=spam_china_crop_list[id]
    multi_year_bool=multi_year_data_training[id]

    spam_list_len=len(spam_list)
    if spam_list_len!=1:
      continue

    SPAM_shortName=spam_list[0]

    if multi_year_bool==0:
      continue

    train_year_list=train_year_list_list[id]
    crop_id+=1
    crop_name=SPAM_shortName

    '''
    if crop_name!='soyb':
      continue
    '''

    indicator_test=merge_indicator_band_as_one_image_RF(crop_name,predict_year)#.addBands(test_map)
    '''
    for train_year in train_year_list:
      #spam_a=ee.ImageCollection('projects/ee-global-crop/assets/CDL_CHN_multiyear_base_layer_10km_1').filter(ee.Filter.stringContains('system:index',crop_name)).filter(ee.Filter.stringContains('system:index',str(train_year))).first()
      spam_a=ee.Image("projects/ee-global-crop/assets/CDL_CHN_multiyear_base_layer_10km/"+crop_name+'_'+str(train_year))

      train_map=spam_a#.divide(pixelArea_10km).multiply(10000)
      #train_map=pixelArea_10km.multiply(train_map).divide(10000)

      train_map=train_map.rename('crop_area_prop_1').updateMask(train_map).unmask(0)

      indicator_train=merge_indicator_band_as_one_image_RF(crop_name,train_year).addBands(train_map)

      sample=ee.FeatureCollection('projects/ee-global-crop-1/assets/Crop_sample_CHN_multiYear_1/'+crop_name+'_'+str(train_year)).filter(ee.Filter.eq('crop_area_prop',1))
      indicator_f=indicator_train.sampleRegions(**{
        'collection':sample,
        #properties:,
        'scale':10000,
        #projection:,
        #'tileScale':16,
        'geometries':True
      })
      if train_year==train_year_list[0]:
        indicator_f_all_0=indicator_f
      else:
        indicator_f_all_0=indicator_f_all_0.merge(indicator_f)

    task1=ee.batch.Export.table.toAsset(**{
      'collection':indicator_f_all_0,
      'description':'indicator_'+crop_name,
      'assetId':'projects/ee-global-crop-1/assets/Crop_sample_CHN_multiYear_indicator/'+crop_name,
    });
    task1.start()
    '''
    if crop_name!='rape':
      indicator_f_all_0=ee.FeatureCollection('projects/ee-global-crop-1/assets/Crop_sample_CHN_multiYear_indicator/'+crop_name)
      split=10000/len(train_year_list)*10000
      withRandom = indicator_f_all_0.randomColumn('random');
      indicator_f_all = withRandom.filter(ee.Filter.lt('random', split));
    else:
      indicator_f_all = ee.FeatureCollection('projects/ee-global-crop-1/assets/Crop_sample_CHN_multiYear_indicator/'+crop_name+'_subset')

    print(crop_name+':')
    print(indicator_f_all.first().propertyNames().getInfo())
    res=train_RF_and_get_Predict_Result(indicator_f_all,indicator_test)
    '''
    if crop_id==1:
      exp=res.rename(crop_name)
    else:
      exp=exp.addBands(res.rename(crop_name))
    '''
    exp=res.rename(crop_name)

    task1=ee.batch.Export.image.toAsset(**{
      'image':exp,
      'description':"prob"+"_predict_"+str(predict_year)+'_'+crop_name,
      'assetId':"projects/ee-global-crop-1/assets/Crop_prob_CHN_RF_multiYear_1/"+"prob"+"_predict_"+str(predict_year)+'_'+crop_name,
      'region':chn_adm1.geometry(),
      'scale':10000,
    })
    task1.start()

def train_RF_and_get_Predict_Result(sampleData,input_img):
  withRandom = sampleData.randomColumn('random');

  split = 0.7;
  training = withRandom.filter(ee.Filter.lt('random', split));
  testing = withRandom.filter(ee.Filter.gte('random', split));

  inputProperties = input_img.bandNames().remove('crop_area_prop_1')#sampleData.first().propertyNames().remove('change_area').remove('system:index');
  classProperties = 'crop_area_prop_1';

  #build classifier
  classifier = ee.Classifier.smileRandomForest(100).setOutputMode('REGRESSION').train(**{
    'features': training,
    'classProperty': classProperties,
    'inputProperties' : inputProperties,
  });

  input_img=input_img.select(inputProperties)
  res=input_img.classify(classifier)

  return res

In [ ]:
export_indicator_sample_collection_RF_multiYear(predict_year=2010)

In [ ]:
CDL_stratifiedSample(2010)

In [ ]:
for year in range(1980,2023):
  if year%5==0:
    continue
  if year==2010:
    continue
  #export_indicator_sample_collection_RF(train_year=2010,predict_year=year)
  export_indicator_sample_collection_RF_multiYear(predict_year=year)

# **USA_RF_expain**

In [ ]:
def train_RF_and_get_explain(sampleData):
  withRandom = sampleData.randomColumn('random');

  split = 0.7;
  training = withRandom.filter(ee.Filter.lt('random', split));
  testing = withRandom.filter(ee.Filter.gte('random', split));

  inputProperties = sampleData.first().propertyNames().remove('crop_area_prop_1').remove('system:index').remove('crop_area_prop')#sampleData.first().propertyNames().remove('change_area').remove('system:index');
  classProperties = 'crop_area_prop_1';

  #build classifier
  classifier = ee.Classifier.smileRandomForest(100).setOutputMode('REGRESSION').train(**{
    'features': training,
    'classProperty': classProperties,
    'inputProperties' : inputProperties,
  });
  dic=classifier.explain()


  return dic

def export_indicator_explain_RF(train_year):
  crop_type_list=crop_type_replace['crop_id'].tolist()
  exp_c=ee.FeatureCollection([])
  for crop_id in crop_type_list:
    crop_type_selected=crop_type_replace.loc[crop_type_replace['crop_id'] == crop_id]
    SPAM_shortName=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("SPAM_shortName")])
    crop_name=SPAM_shortName
    id=crop_name_list.index(SPAM_shortName)
    USDA_name=usda_name_list[id]
    if USDA_name=='BEANS':
      continue
    if USDA_name !='':
      train_map=ee.Image("projects/ee-global-crop/assets/CDL_10km_area/CDL_10km_area_p_"+str(train_year)+'_'+USDA_name)
      #train_map=pixelArea_10km.multiply(train_map).divide(10000)

      train_map=train_map.rename('crop_area_prop_1')
      #train_map=train_map.divide(pixelArea_10km).rename('crop_area_prop')#transfer abosulute value into prop

      #test_map=ee.Image("projects/ee-global-crop/assets/CDL_10km_area/CDL_10km_area_p_"+str(predict_year)+'_'+USDA_name)
      ##test_map=pixelArea_10km.multiply(test_map).divide(10000)

      #test_map=test_map.rename('crop_area_prop_1')
      ##test_map=test_map.divide(pixelArea_10km).rename('crop_area_prop')#transfer abosulute value into prop

      indicator_train=merge_indicator_band_as_one_image_RF(crop_name,train_year).addBands(train_map)
      #indicator_test=merge_indicator_band_as_one_image_RF(crop_name,predict_year)#.addBands(test_map)
      sample=ee.FeatureCollection('projects/ee-global-crop/assets/CDL_sample_3/'+crop_name+'_'+str(train_year)).filter(ee.Filter.eq('crop_area_prop',1))
      indicator_f=indicator_train.sampleRegions(**{
        'collection':sample,
        #properties:,
        'scale':10000,
        #projection:,
        #'tileScale':16,
        #geometries:
      })
      print(crop_name+':')
      print(indicator_f.first().propertyNames().getInfo())
      exp=train_RF_and_get_explain(indicator_f)#train_RF_and_get_Predict_Result(indicator_f,indicator_test)

      exp_c=ee.FeatureCollection(ee.Feature(None).set(exp).set('crop_name',crop_name))
      task1=ee.batch.Export.table.toDrive(**{
        'collection':exp_c,
        'description':"RF_explain_"+"train_"+str(train_year),
        'folder':"RF_explain",
        'fileFormat':'csv',
      })
      task1.start()

In [ ]:
export_indicator_explain_RF(2010)

# **AF_RF_explain**

In [ ]:
def train_RF_and_get_explain(sampleData):
  withRandom = sampleData.randomColumn('random');

  split = 0.7;
  training = withRandom.filter(ee.Filter.lt('random', split));
  testing = withRandom.filter(ee.Filter.gte('random', split));

  inputProperties = sampleData.first().propertyNames().remove('crop_area_prop_1').remove('system:index').remove('crop_area_prop')#sampleData.first().propertyNames().remove('change_area').remove('system:index');
  classProperties = 'crop_area_prop_1';

  #build classifier
  classifier = ee.Classifier.smileRandomForest(100).setOutputMode('REGRESSION').train(**{
    'features': training,
    'classProperty': classProperties,
    'inputProperties' : inputProperties,
  });
  dic=classifier.explain()


  return dic

def export_indicator_explain_RF(train_year):
  crop_type_list=crop_type_replace['crop_id'].tolist()
  exp_c=ee.FeatureCollection([])
  for crop_id in crop_type_list:
    crop_type_selected=crop_type_replace.loc[crop_type_replace['crop_id'] == crop_id]
    SPAM_shortName=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("SPAM_shortName")])
    crop_name=SPAM_shortName

    spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2010/spam2010V2r0_global_H_'+SPAM_shortName.upper()+'_A').clip(africa)
    spam_a=spam_a.updateMask(spam_a.neq(-1)).unmask(0)
    train_map=spam_a.divide(pixelArea_10km).multiply(10000)


    train_map=train_map.rename('crop_area_prop_1')
    #train_map=train_map.divide(pixelArea_10km).rename('crop_area_prop')#transfer abosulute value into prop


    indicator_train=merge_indicator_band_as_one_image_RF(crop_name,train_year).addBands(train_map)
    #indicator_test=merge_indicator_band_as_one_image_RF(crop_name,predict_year)#.addBands(test_map)
    sample=ee.FeatureCollection('projects/ee-global-crop/assets/Crop_sample_AF/'+crop_name+'_'+str(train_year)).filter(ee.Filter.eq('crop_area_prop',1))
    indicator_f=indicator_train.sampleRegions(**{
      'collection':sample,
      #properties:,
      'scale':10000,
      #projection:,
      #'tileScale':16,
      #geometries:
    })
    print(crop_name+':')
    print(indicator_f.first().propertyNames().getInfo())
    exp=train_RF_and_get_explain(indicator_f)#train_RF_and_get_Predict_Result(indicator_f,indicator_test)

    exp_c=ee.FeatureCollection(ee.Feature(None).set(exp).set('crop_name',crop_name))
    task1=ee.batch.Export.table.toDrive(**{
      'collection':exp_c,
      'description':"RF_explain_"+"train_"+str(train_year),
      'folder':"RF_explain_AF",
      'fileFormat':'csv',
    })
    task1.start()

In [ ]:
export_indicator_explain_RF(train_year=2010)

# **CHN_RF_explain**

In [ ]:
def train_RF_and_get_explain(sampleData):
  withRandom = sampleData.randomColumn('random');

  split = 0.7;
  training = withRandom.filter(ee.Filter.lt('random', split));
  testing = withRandom.filter(ee.Filter.gte('random', split));

  inputProperties = sampleData.first().propertyNames().remove('crop_area_prop_1').remove('system:index').remove('crop_area_prop')#sampleData.first().propertyNames().remove('change_area').remove('system:index');
  classProperties = 'crop_area_prop_1';

  #build classifier
  classifier = ee.Classifier.smileRandomForest(100).setOutputMode('REGRESSION').train(**{
    'features': training,
    'classProperty': classProperties,
    'inputProperties' : inputProperties,
  });
  dic=classifier.explain()


  return dic

def export_indicator_explain_RF(train_year):
  crop_id=0
  for crop_name in spam_china_crop_list_1:
    if crop_name=='pige'or crop_name=='yams'or crop_name=='chic' or crop_name=='cowp':
      continue
    crop_id+=1
    SPAM_shortName=crop_name

    if train_year==2010:
      spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2010/spam2010V2r0_global_H_'+SPAM_shortName.upper()+'_A').clip(chn_adm1)
      spam_a=spam_a.updateMask(spam_a.gte(0)).unmask(0)
    elif train_year==2005:
      spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2005/SPAM2005V3r2_global_H_T'+'A_'+SPAM_shortName.upper()+'_A').clip(chn_adm1)
      spam_a=spam_a.updateMask(spam_a.gte(0)).unmask(0)

    train_map=spam_a.divide(pixelArea_10km).multiply(10000)
    #train_map=pixelArea_10km.multiply(train_map).divide(10000)

    train_map=train_map.rename('crop_area_prop_1')

    indicator_train=merge_indicator_band_as_one_image_RF(crop_name,train_year).addBands(train_map)
    #indicator_test=merge_indicator_band_as_one_image_RF(crop_name,predict_year)#.addBands(test_map)
    sample=ee.FeatureCollection('projects/ee-global-crop-1/assets/Crop_sample_CHN/'+crop_name+'_'+str(train_year)).filter(ee.Filter.eq('crop_area_prop',1))
    indicator_f=indicator_train.sampleRegions(**{
      'collection':sample,
      #properties:,
      'scale':10000,
      #projection:,
      #'tileScale':16,
      #geometries:
    })
    print(crop_name+':')
    print(indicator_f.first().propertyNames().getInfo())

    exp=train_RF_and_get_explain(indicator_f)#train_RF_and_get_Predict_Result(indicator_f,indicator_test)

    exp_c=ee.FeatureCollection(ee.Feature(None).set(exp).set('crop_name',crop_name))
    task1=ee.batch.Export.table.toDrive(**{
      'collection':exp_c,
      'description':"RF_explain_"+"train_"+str(train_year),
      'folder':"RF_explain_CHN",
      'fileFormat':'csv',
    })
    task1.start()

In [ ]:
def export_indicator_sample_collection_RF_multiYear():
  crop_id=0
  for crop_china_name in china_crop_list:
    #crop_name=china_crop_list[id]
    id=china_crop_list.index(crop_china_name)
    spam_list=spam_china_crop_list[id]
    multi_year_bool=multi_year_data_training[id]

    spam_list_len=len(spam_list)
    if spam_list_len!=1:
      continue

    SPAM_shortName=spam_list[0]

    if multi_year_bool==0:
      continue

    train_year_list=train_year_list_list[id]
    crop_id+=1
    crop_name=SPAM_shortName

    if crop_name!='rape':
      indicator_f_all_0=ee.FeatureCollection('projects/ee-global-crop-1/assets/Crop_sample_CHN_multiYear_indicator/'+crop_name)
      split=10000/len(train_year_list)*10000
      withRandom = indicator_f_all_0.randomColumn('random');
      indicator_f_all = withRandom.filter(ee.Filter.lt('random', split));
    else:
      indicator_f_all = ee.FeatureCollection('projects/ee-global-crop-1/assets/Crop_sample_CHN_multiYear_indicator/'+crop_name+'_subset')

    print(crop_name+':')
    print(indicator_f_all.first().propertyNames().getInfo())
    exp=train_RF_and_get_explain(indicator_f_all)
    '''
    if crop_id==1:
      exp=res.rename(crop_name)
    else:
      exp=exp.addBands(res.rename(crop_name))
    '''
    exp_c=ee.FeatureCollection(ee.Feature(None).set(exp).set('crop_name',crop_name))
    task1=ee.batch.Export.table.toDrive(**{
      'collection':exp_c,
      'description':"RF_explain_"+crop_name,
      'folder':"RF_explain_CHN_multi",
      'fileFormat':'csv',
    })
    task1.start()

In [ ]:
export_indicator_sample_collection_RF_multiYear()

soyb:
['random', 'rurc', 'field_size', 'latitude', 'soil_pH', 'soil_sand', 'crop_area_prop', 'prec', 'soil_texture', 'system_share_r', 'cropland', 'radi_down', 'longitude', 'elevation', 'temp', 'soil_clay', 'system_share_i', 'suit_l', 'gdd', 'crop_area_prop_1', 'suit_i', 'suit_h', 'evap_veg', 'soil_water', 'slope', 'soil_orgnC', 'yield_h', 'yield_i', 'yield_l', 'system:index']
rice:
['random', 'rurc', 'field_size', 'latitude', 'soil_pH', 'soil_sand', 'crop_area_prop', 'prec', 'soil_texture', 'system_share_r', 'cropland', 'radi_down', 'longitude', 'elevation', 'temp', 'soil_clay', 'system_share_i', 'suit_l', 'gdd', 'crop_area_prop_1', 'suit_i', 'suit_h', 'evap_veg', 'soil_water', 'slope', 'soil_orgnC', 'yield_h', 'yield_i', 'yield_l', 'system:index']
sugc:
['random', 'rurc', 'field_size', 'latitude', 'soil_pH', 'soil_sand', 'crop_area_prop', 'prec', 'soil_texture', 'system_share_r', 'cropland', 'radi_down', 'longitude', 'elevation', 'temp', 'soil_clay', 'system_share_i', 'suit_l', 'gdd'

In [ ]:
export_indicator_explain_RF(train_year=2010)

soyb:
['system:index', 'crop_area_prop', 'rurc', 'field_size', 'latitude', 'soil_pH', 'soil_sand', 'prec', 'soil_texture', 'system_share_r', 'cropland', 'radi_down', 'longitude', 'elevation', 'temp', 'soil_clay', 'system_share_i', 'suit_l', 'gdd', 'crop_area_prop_1', 'suit_i', 'suit_h', 'evap_veg', 'soil_water', 'slope', 'soil_orgnC', 'yield_h', 'yield_i', 'yield_l']
rice:
['system:index', 'crop_area_prop', 'rurc', 'field_size', 'latitude', 'soil_pH', 'soil_sand', 'prec', 'soil_texture', 'system_share_r', 'cropland', 'radi_down', 'longitude', 'elevation', 'temp', 'soil_clay', 'system_share_i', 'suit_l', 'gdd', 'crop_area_prop_1', 'suit_i', 'suit_h', 'evap_veg', 'soil_water', 'slope', 'soil_orgnC', 'yield_h', 'yield_i', 'yield_l']
bean:
['system:index', 'crop_area_prop', 'rurc', 'field_size', 'latitude', 'soil_pH', 'soil_sand', 'prec', 'soil_texture', 'system_share_r', 'cropland', 'radi_down', 'longitude', 'elevation', 'temp', 'soil_clay', 'system_share_i', 'suit_l', 'gdd', 'crop_area_p

# **Universal_export_functions**

In [ ]:
def export_crop_intensity_as_10km(year):
  if year<=2019 and year>=2001:
    GCI_collection=ee.ImageCollection('projects/ee-global-crop/assets/Crop_intensity_GCI_2000-19')
    GCI_m=GCI_collection.median()
    p=GCI_collection.first().projection()
    GCI=ee.Image('projects/ee-global-crop/assets/Crop_intensity_GCI_2000-19/'+str(year))
    GCI=GCI.updateMask(GCI).unmask(GCI_m)
    cropland=get_cropland_layer(year)
    #GCI=GCI.updateMask(cropland).unmask(1)
    GCI=GCI#.updateMask(cropland)

    GCI_0=GCI.eq(0)
    GCI_1=GCI.eq(1)
    GCI_2=GCI.eq(2)
    GCI_3=GCI.eq(3)

    def reduceResolution_10km(img,p):

      img=img.setDefaultProjection(**{
        'crs': p,
        'crsTransform':None,
        'scale':250
      })

      img_10km = img.reduceResolution(**{
        'reducer': ee.Reducer.mean(),
        'bestEffort':True,
        'maxPixels':60000
      }).reproject(p,None,10000).rename('cropIntensity')

      return img_10km

    GCI_0_10km=reduceResolution_10km(GCI_0,p)
    GCI_1_10km=reduceResolution_10km(GCI_1,p)
    GCI_2_10km=reduceResolution_10km(GCI_2,p)
    GCI_3_10km=reduceResolution_10km(GCI_3,p)

    GCI_10km=GCI_1_10km.multiply(1).add(GCI_2_10km.multiply(2)).add(GCI_3_10km.multiply(3)).divide(GCI_1_10km.add(GCI_2_10km).add(GCI_3_10km))

    global_g = ee.Geometry.Polygon([[-180,-90],[-180,90],[180,-90],[180,90]],None,False)
    task1=ee.batch.Export.image.toAsset(**{
      'image':GCI_10km,
      'description':"GCI_10km"+'_'+str(year),
      'assetId':"projects/ee-global-crop/assets/Crop_intensity_GCI_2000-19_10km_new/"+"GCI_10km"+'_'+str(year),
      'region':global_g,
      'scale':10000,
    })
    task1.start()
  else:
    print('inter yeasr between 2001 to 2019!')


# **USA_export_functions**

In [ ]:
def adjust_cropland_max_by_base_crop_map(year):
  max=get_harvest_area_max(year)
  #global_g = ee.Geometry.Polygon([[-180,-90],[-180,90],[180,-90],[180,90]],None,False)
  spam2010_table=pd.read_csv('/content/drive/MyDrive/global_crop_dataset/CDL2010_crop_area_table-USA.csv')
  adm_list=spam2010_table['ADM1_NAME'].values.tolist()

  crop_type_list=crop_type_replace['crop_id'].tolist()

  crop_map_all=ee.ImageCollection([])

  GEO_count=-1
  for GEO_name in adm_list:
    GEO_count+=1
    #spam2010_table=pd.read_csv('/content/drive/MyDrive/spam2010_suit-africa/spam2010_suit_'+str(GEO_count))

    if GEO_name=='Alaska':
      continue
    if GEO_count!=28:
      continue

    geo_adm=usa_adm1
    geo_bound=geo_adm.filter(ee.Filter.eq("ADM1_NAME",GEO_name))

    task1=ee.batch.Export.image.toDrive(**{
      'image':max.clip(geo_bound),
      'description':"max_GEO_"+str(GEO_count)+'_'+str(year),
      'folder':"Crop_area_max_USA",
      'region':geo_bound.geometry(),
      'scale':10000,
    })
    task1.start()

In [ ]:
def export_base_map_by_adm1(year):
  spam2010_table=pd.read_csv('/content/drive/MyDrive/global_crop_dataset/CDL2010_crop_area_table-USA.csv')
  adm_list=spam2010_table['ADM1_NAME'].values.tolist()
  crop_type_list=crop_type_replace['crop_id'].tolist()

  GEO_count=-1
  for GEO_name in adm_list:
    GEO_count+=1
    #spam2010_table=pd.read_csv('/content/drive/MyDrive/spam2010_suit-africa/spam2010_suit_'+str(GEO_count))

    if GEO_name=='Alaska':
      continue

    geo_adm=usa_adm1
    geo_bound=geo_adm.filter(ee.Filter.eq("ADM1_NAME",GEO_name))

    for crop_id in crop_type_list:
      crop_type_selected=crop_type_replace.loc[crop_type_replace['crop_id'] == crop_id]
      SPAM_shortName=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("SPAM_shortName")])

      id=crop_name_list.index(SPAM_shortName)
      USDA_name=usda_name_list[id]

      if USDA_name in USDA_name_list:
        spam_a=ee.Image("projects/ee-global-crop/assets/CDL_10km_area/CDL_10km_area_p_"+str(year)+'_'+USDA_name)#replace spam_a with cdl_2010 10km
        pixelArea_10km=ee.Image('projects/ee-global-crop/assets/pixelArea_global')
        spam_a=pixelArea_10km.multiply(spam_a).divide(10000).toFloat()
      else:
        spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2010/spam2010V2r0_global_H_'+SPAM_shortName.upper()+'_A')
        spam_a=spam_a.updateMask(spam_a.neq(-1)).unmask(0).toFloat()

      #transform area as prop
      pixelArea_10km=ee.Image('projects/ee-global-crop/assets/pixelArea_global')
      spam_a=spam_a.multiply(10000).divide(pixelArea_10km).toFloat()

      if crop_id==1:
        spam_a_all=spam_a
      else:
        spam_a_all=spam_a_all.addBands(spam_a)

    task1=ee.batch.Export.image.toDrive(**{
      'image':spam_a_all.clip(geo_bound),
      'description':"CDL_GEO_"+str(GEO_count)+'_'+str(year),
      'folder':"Crop_map_USA_CDL_refer",
      'region':geo_bound.geometry(),
      'scale':10000,
    })
    task1.start()

In [ ]:
for year in range(2008,2023):
  if year!=2010:
    export_base_map_by_adm1(year=year)

In [ ]:
def export_pixelArea_10km_by_adm1():
  spam2010_table=pd.read_csv('/content/drive/MyDrive/global_crop_dataset/CDL2010_crop_area_table-USA.csv')
  adm_list=spam2010_table['ADM1_NAME'].values.tolist()
  crop_type_list=crop_type_replace['crop_id'].tolist()

  GEO_count=-1
  for GEO_name in adm_list:
    GEO_count+=1
    #spam2010_table=pd.read_csv('/content/drive/MyDrive/spam2010_suit-africa/spam2010_suit_'+str(GEO_count))

    if GEO_name=='Alaska':
      continue

    geo_adm=usa_adm1
    geo_bound=geo_adm.filter(ee.Filter.eq("ADM1_NAME",GEO_name))

    pixelArea_10km=ee.Image('projects/ee-global-crop/assets/pixelArea_global')

    task1=ee.batch.Export.image.toDrive(**{
      'image':pixelArea_10km,#.clip(geo_bound),
      'description':"pixelArea_10km_"+str(GEO_count),
      'folder':"pixelArea_10km_USA",
      'region':geo_bound.geometry(),
      'scale':10000,
    })
    task1.start()

In [ ]:
export_pixelArea_10km_by_adm1()

In [ ]:
#define functions to make allocation
def Export_allRasterLayer_toDrive(year):
  spam2010_table=pd.read_csv('/content/drive/MyDrive/global_crop_dataset/CDL2010_crop_area_table-USA.csv')
  adm_list=spam2010_table['ADM1_NAME'].values.tolist()
  crop_type_list=crop_type_replace['crop_id'].tolist()

  GEO_count=-1
  for GEO_name in adm_list:
    GEO_count+=1
  #spam2010_table=pd.read_csv('/content/drive/MyDrive/spam2010_suit-africa/spam2010_suit_'+str(GEO_count))

    if GEO_name=='Alaska':
      continue

    geo_adm=usa_adm1
    geo_bound=geo_adm.filter(ee.Filter.eq("ADM1_NAME",GEO_name))

    for crop_id in crop_type_list:
      crop_type_selected=crop_type_replace.loc[crop_type_replace['crop_id'] == crop_id]
      SPAM_shortName=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("SPAM_shortName")])
      SPAM_shortName_2000=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("SPAM2000_shortName")])
      crop_name=SPAM_shortName
      id=crop_name_list.index(crop_name)
      USDA_name=usda_name_list[id]

      if USDA_name in USDA_name_list:
        spam_a=ee.Image("projects/ee-global-crop/assets/CDL_10km_area/CDL_10km_area_p_"+str(year)+'_'+USDA_name)#replace spam_a with cdl_2010 10km
      else:
        spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2010/spam2010V2r0_global_H_'+SPAM_shortName.upper()+'_A')
        pixelArea_10km=ee.Image('projects/ee-global-crop/assets/pixelArea_global')
        spam_a=spam_a.multiply(10000).divide(pixelArea_10km)

      suit_i=produce_suit_layer(id+1,year,'i').rename('suit_i_'+crop_name).toFloat()
      suit_h=produce_suit_layer(id+1,year,'h').rename('suit_h_'+crop_name).toFloat()
      suit_l=produce_suit_layer(id+1,year,'l').rename('suit_l_'+crop_name).toFloat()

      potYield_i=produce_PotYield_layer(id+1,year,'i').rename('yield_i_'+crop_name).toFloat()
      potYield_h=produce_PotYield_layer(id+1,year,'h').rename('yield_h_'+crop_name).toFloat()
      potYield_l=produce_PotYield_layer(id+1,year,'l').rename('yield_l_'+crop_name).toFloat()
      indicator=suit_i.addBands(suit_h).addBands(suit_l).addBands(potYield_i).addBands(potYield_h).addBands(potYield_l).addBands(spam_a.toFloat())


      if crop_id==1:
        indicator_all=indicator
      else:
        indicator_all=indicator_all.addBands(indicator)

    indicator_all=indicator_all.addBands(merge_indicator_band_as_one_image_RF(year).toFloat())
  #print(indicator_all.bandNames().getInfo())

    task1=ee.batch.Export.image.toDrive(**{
      'image':indicator_all.clip(geo_bound),
      'description':"Indicator_GEO_"+str(GEO_count)+'_'+str(year),
      'folder':"Indicator_USA",
      'region':geo_bound.geometry(),
      'scale':10000,
    })
    task1.start()


In [ ]:
for year in range(2008,2023):
  Export_allRasterLayer_toDrive(year)

# **AF_export_functions**

In [ ]:
def adjust_cropland_max_by_base_crop_map(year):
  max=get_harvest_area_max(year)
  #global_g = ee.Geometry.Polygon([[-180,-90],[-180,90],[180,-90],[180,90]],None,False)
  spam2010_table=pd.read_csv('/content/drive/MyDrive/global_crop_dataset/SPAM2010_crop_area_table-africa.csv')
  adm_list=spam2010_table['ADM0_NAME'].values.tolist()

  crop_type_list=crop_type_replace['crop_id'].tolist()

  crop_map_all=ee.ImageCollection([])

  GEO_count=-1
  for GEO_name in adm_list:
    GEO_count+=1
    #spam2010_table=pd.read_csv('/content/drive/MyDrive/spam2010_suit-africa/spam2010_suit_'+str(GEO_count))
    if GEO_count!=3:
      continue
    geo_adm=geo_adm0
    geo_bound=geo_adm.filter(ee.Filter.eq("ADM0_NAME",GEO_name))

    task1=ee.batch.Export.image.toDrive(**{
      'image':max.clip(geo_bound),
      'description':"max_GEO_"+str(GEO_count)+'_'+str(year),
      'folder':"Crop_area_max_Africa",
      'region':geo_bound.geometry(),
      'scale':10000,
    })
    task1.start()

In [ ]:
#define functions to make allocation
def Export_pixelArea_toDrive():
  spam2010_table=pd.read_csv('/content/drive/MyDrive/global_crop_dataset/SPAM2010_crop_area_table-africa.csv')
  adm_list=spam2010_table['ADM0_NAME'].values.tolist()
  crop_type_list=crop_type_replace['crop_id'].tolist()

  GEO_count=-1
  for GEO_name in adm_list:
    GEO_count+=1
  #spam2010_table=pd.read_csv('/content/drive/MyDrive/spam2010_suit-africa/spam2010_suit_'+str(GEO_count))

    if GEO_name=="R¨¦union":
      GEO_name="Réunion"
    if GEO_name=="C?te d'Ivoire":
      GEO_name="Côte d'Ivoire"

    geo_adm=geo_adm0
    geo_bound=geo_adm.filter(ee.Filter.eq("ADM0_NAME",GEO_name))

    #transform area as prop
    pixelArea_10km=ee.Image('projects/ee-global-crop/assets/pixelArea_global')

    task1=ee.batch.Export.image.toDrive(**{
      'image':pixelArea_10km,
      'description':"pixelArea_10km_"+str(GEO_count),
      'folder':"pixelArea_10km_AF",
      'region':geo_bound.geometry(),
      'scale':10000,
    })
    task1.start()

In [ ]:
Export_pixelArea_toDrive()

In [ ]:
#define functions to make allocation
def Export_spamBaseMap_toDrive(year):
  spam2010_table=pd.read_csv('/content/drive/MyDrive/global_crop_dataset/SPAM2010_crop_area_table-africa.csv')
  adm_list=spam2010_table['ADM0_NAME'].values.tolist()
  crop_type_list=crop_type_replace['crop_id'].tolist()

  GEO_count=-1
  for GEO_name in adm_list:
    GEO_count+=1
  #spam2010_table=pd.read_csv('/content/drive/MyDrive/spam2010_suit-africa/spam2010_suit_'+str(GEO_count))

    if GEO_name=="R¨¦union":
      GEO_name="Réunion"
    if GEO_name=="C?te d'Ivoire":
      GEO_name="Côte d'Ivoire"

    geo_adm=geo_adm0
    geo_bound=geo_adm.filter(ee.Filter.eq("ADM0_NAME",GEO_name))

    for crop_id in crop_type_list:
      crop_type_selected=crop_type_replace.loc[crop_type_replace['crop_id'] == crop_id]
      SPAM_shortName=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("SPAM_shortName")])

      spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2010/spam2010V2r0_global_H_'+SPAM_shortName.upper()+'_A')
      spam_a=spam_a.updateMask(spam_a.neq(-1)).unmask(0).toFloat()

      #transform area as prop
      pixelArea_10km=ee.Image('projects/ee-global-crop/assets/pixelArea_global')
      spam_a=spam_a.multiply(10000).divide(pixelArea_10km).toFloat()

      if crop_id==1:
        spam_a_all=spam_a
      else:
        spam_a_all=spam_a_all.addBands(spam_a)

    task1=ee.batch.Export.image.toDrive(**{
      'image':spam_a_all.clip(geo_bound),
      'description':"SPAM_GEO_"+str(GEO_count)+'_'+str(year),
      'folder':"Crop_map_AF_SPAM_2010",
      'region':geo_bound.geometry(),
      'scale':10000,
    })
    task1.start()

In [ ]:
Export_pixelArea_toDrive()

In [ ]:
Export_spamBaseMap_toDrive(year=2010)

In [ ]:
#define functions to make allocation
def Export_allRasterLayer_toDrive(year):
  spam2010_table=pd.read_csv('/content/drive/MyDrive/global_crop_dataset/SPAM2010_crop_area_table-africa.csv')
  adm_list=spam2010_table['ADM0_NAME'].values.tolist()
  crop_type_list=crop_type_replace['crop_id'].tolist()

  GEO_count=-1
  for GEO_name in adm_list:
    GEO_count+=1
    if GEO_count!=28:
      continue
  #spam2010_table=pd.read_csv('/content/drive/MyDrive/spam2010_suit-africa/spam2010_suit_'+str(GEO_count))

    if GEO_name=="R¨¦union":
      GEO_name="Réunion"
    if GEO_name=="C?te d'Ivoire":
      GEO_name="Côte d'Ivoire"

    geo_adm=geo_adm0
    geo_bound=geo_adm.filter(ee.Filter.eq("ADM0_NAME",GEO_name))

    for crop_id in crop_type_list:
      crop_type_selected=crop_type_replace.loc[crop_type_replace['crop_id'] == crop_id]
      SPAM_shortName=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("SPAM_shortName")])
      SPAM_shortName_2000=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("SPAM2000_shortName")])
      crop_name=SPAM_shortName
      id=crop_name_list.index(crop_name)

      spam_10=ee.Image('projects/ee-global-crop/assets/SPAM2010/spam2010V2r0_global_H_'+SPAM_shortName.upper()+'_A')
      spam_10=spam_10.updateMask(spam_10.gte(0)).unmask(0)

      #transform area as prop
      pixelArea_10km=ee.Image('projects/ee-global-crop/assets/pixelArea_global').rename('pixelArea_10km')
      spam_10=spam_10.multiply(10000).divide(pixelArea_10km).toFloat().rename('spam_2010_'+crop_name)

      spam_05=ee.Image('projects/ee-global-crop/assets/SPAM2005/SPAM2005V3r2_global_H_T'+'A_'+crop_name.upper()+'_A')
      spam_05=spam_05.updateMask(spam_05.gte(0)).unmask(0)
      spam_05=spam_05.divide(pixelArea_10km).multiply(10000).toFloat().rename('spam_2005_'+crop_name)

      spam_17=ee.Image("projects/ee-global-crop/assets/SPAM2017/"+'spam2017V2r1_SSA_H'+'_'+crop_name.upper()+'_'+'A')
      spam_17=spam_17.updateMask(spam_17.gte(0)).unmask(0)
      spam_17=spam_17.divide(pixelArea_10km).multiply(10000).toFloat().rename('spam_2017_'+crop_name)

      spam_00=ee.Image("projects/ee-global-crop/assets/SPAM2000/"+'spam2000V3r107_global_'+'H_'+SPAM_shortName_2000.upper()+'_'+'A')
      spam_00=spam_00.updateMask(spam_00.gte(0)).unmask(0)
      spam_00=spam_00.divide(pixelArea_10km).multiply(10000).toFloat().rename('spam_2000_'+crop_name)

      suit_i=produce_suit_layer(id+1,year,'i').rename('suit_i_'+crop_name).toFloat()
      suit_h=produce_suit_layer(id+1,year,'h').rename('suit_h_'+crop_name).toFloat()
      suit_l=produce_suit_layer(id+1,year,'l').rename('suit_l_'+crop_name).toFloat()

      potYield_i=produce_PotYield_layer(id+1,year,'i').rename('yield_i_'+crop_name).toFloat()
      potYield_h=produce_PotYield_layer(id+1,year,'h').rename('yield_h_'+crop_name).toFloat()
      potYield_l=produce_PotYield_layer(id+1,year,'l').rename('yield_l_'+crop_name).toFloat()
      indicator=suit_i.addBands(suit_h).addBands(suit_l).addBands(potYield_i).addBands(potYield_h).addBands(potYield_l).addBands(spam_10).addBands(spam_05).addBands(spam_17).addBands(spam_00)


      if crop_id==1:
        indicator_all=indicator
      else:
        indicator_all=indicator_all.addBands(indicator)

    indicator_all=indicator_all.addBands(merge_indicator_band_as_one_image_RF(year).toFloat())
  #print(indicator_all.bandNames().getInfo())


    task1=ee.batch.Export.image.toDrive(**{
      'image':indicator_all.clip(geo_bound),
      'description':"Indicator_GEO_"+str(GEO_count)+'_'+str(year),
      'folder':"Indicator_AF",
      'region':geo_bound.geometry(),
      'scale':10000,
    })
    task1.start()


In [ ]:
for year in range(1961,2023,1):
  Export_allRasterLayer_toDrive(year)

# **CHN_export_functions**

In [ ]:
def adjust_cropland_max_by_base_crop_map(year):
  max=get_harvest_area_max(year)
  #global_g = ee.Geometry.Polygon([[-180,-90],[-180,90],[180,-90],[180,90]],None,False)
  spam2010_table=pd.read_csv('/content/drive/MyDrive/global_crop_dataset/SPAM2010_crop_area_table-CHN.csv')
  adm_list=spam2010_table['ADM1_NAME'].values.tolist()

  crop_map_all=ee.ImageCollection([])

  GEO_count=-1
  for GEO_name in adm_list:
    GEO_count+=1
    if GEO_count!=28:
      continue
    #spam2010_table=pd.read_csv('/content/drive/MyDrive/spam2010_suit-africa/spam2010_suit_'+str(GEO_count))
    geo_adm=chn_adm1
    geo_bound=geo_adm.filter(ee.Filter.eq("ADM1_NAME",GEO_name))

    task1=ee.batch.Export.image.toDrive(**{
      'image':max.clip(geo_bound),
      'description':"max_GEO_"+str(GEO_count)+'_'+str(year),
      'folder':"Crop_area_max_CHN",
      'region':geo_bound.geometry(),
      'scale':10000,
    })
    task1.start()

In [ ]:
for year in range(2001,2020,1):#2001,2020
  if year==0:
    continue
  adjust_cropland_max_by_base_crop_map(year)

In [ ]:
#define functions to make allocation
def Export_spamBaseMap_toDrive():
  spam2010_table=pd.read_csv('/content/drive/MyDrive/global_crop_dataset/SPAM2010_crop_area_table-CHN.csv')
  adm_list=spam2010_table['ADM1_NAME'].values.tolist()
  crop_type_list=crop_type_replace['crop_id'].tolist()

  GEO_count=-1
  for GEO_name in adm_list:
    GEO_count+=1
  #spam2010_table=pd.read_csv('/content/drive/MyDrive/spam2010_suit-africa/spam2010_suit_'+str(GEO_count))


    geo_adm=chn_adm1
    geo_bound=geo_adm.filter(ee.Filter.eq("ADM1_NAME",GEO_name))

    for crop_id in crop_type_list:
      crop_type_selected=crop_type_replace.loc[crop_type_replace['crop_id'] == crop_id]
      SPAM_shortName=str(crop_type_selected.iloc[0,crop_type_selected.columns.get_loc("SPAM_shortName")])

      spam_a=ee.Image('projects/ee-global-crop/assets/SPAM2010/spam2010V2r0_global_H_'+SPAM_shortName.upper()+'_A')
      spam_a=spam_a.updateMask(spam_a.neq(-1)).unmask(0).toFloat()

      #transform area as prop
      pixelArea_10km=ee.Image('projects/ee-global-crop/assets/pixelArea_global')
      spam_a=spam_a.multiply(10000).divide(pixelArea_10km).toFloat()

      if crop_id==1:
        spam_a_all=spam_a
      else:
        spam_a_all=spam_a_all.addBands(spam_a)

    task1=ee.batch.Export.image.toDrive(**{
      'image':spam_a_all.clip(geo_bound),
      'description':"SPAM_GEO_"+str(GEO_count)+'_'+str(2010),
      'folder':"Crop_map_CHN_SPAM_2010",
      'region':geo_bound.geometry(),
      'scale':10000,
    })
    task1.start()

In [ ]:
Export_spamBaseMap_toDrive()

In [ ]:
#define functions to make allocation
def Export_pixelArea_toDrive():
  spam2010_table=pd.read_csv('/content/drive/MyDrive/global_crop_dataset/SPAM2010_crop_area_table-CHN.csv')
  adm_list=spam2010_table['ADM1_NAME'].values.tolist()
  crop_type_list=crop_type_replace['crop_id'].tolist()

  GEO_count=-1
  for GEO_name in adm_list:
    GEO_count+=1
  #spam2010_table=pd.read_csv('/content/drive/MyDrive/spam2010_suit-africa/spam2010_suit_'+str(GEO_count))

    geo_adm=chn_adm1
    geo_bound=geo_adm.filter(ee.Filter.eq("ADM1_NAME",GEO_name))

    #transform area as prop
    pixelArea_10km=ee.Image('projects/ee-global-crop/assets/pixelArea_global')

    task1=ee.batch.Export.image.toDrive(**{
      'image':pixelArea_10km,
      'description':"pixelArea_10km_"+str(GEO_count),
      'folder':"pixelArea_10km_CHN",
      'region':geo_bound.geometry(),
      'scale':10000,
    })
    task1.start()

In [ ]:
Export_pixelArea_toDrive()